# Install Dependencies
Install additional packages needed to run SQL command in Jupyter cells and send them to a Postgres database server.

## PostgreSQL
Allows for a self-contained instance of PostresSQL for this notebook.

In [ ]:
%mamba install -y -c conda-forge postgresql=17 --quiet
!psql --version

## PostgreSQL Python Extension

In [ ]:
%mamba install -y -c conda-forge postgresql-plpython=17 --quiet

## PostgreSQL Adapter for Python
Essential for connecting and interacting with PostgreSQL databases from Python code in Jupyter notebooks.

In [ ]:
%mamba install -c conda-forge psycopg2 -y --quiet
!python -c "import psycopg2; print(psycopg2.__version__)"

## Language Server Protocol for Jupyter
Enables code completion, hover documentation, and real-time syntax checking for multiple languages in Jupyter notebooks, including SQL.

In [ ]:
%mamba install -c conda-forge jupyterlab-lsp -y --quiet
!jupyter labextension list
!jupyter server extension list

## Language Server for Python
Provides real-time linting, code suggestions, and auto-completion in Jupyter notebooks for Python code.

In [ ]:
%mamba install -c conda-forge python-lsp-server -y --quiet
!pylsp --version

## SQL Magic Extension for Jupyter
Allows running SQL queries directly in Jupyter notebooks using SQL magic (%%sql). Enables seamless execution of SQL queries inside a Jupyter Notebook without switching to a database client.

In [ ]:
%mamba install -c conda-forge jupysql -y --quiet
!python -c "import sql; print(sql.__version__)"
!jupyter labextension list

## SQL Editor Extension
Enables SQL-specific features in Jupyter notebooks, such as:
* Syntax highlighting
* Query auto-completion
* Error checking

In [ ]:
%pip install jupyterlab_sql_editor --quiet
!jupyter labextension list

# Start PostgreSQL

## Define paths and logfile

In [ ]:
import os

DB_DIR = "mylocal_db"
LOGFILE = "postgres_log.txt"
PG_HBA_PATH = os.path.join(DB_DIR, "pg_hba.conf")

## Check PostgreSQL status

In [ ]:
!pg_ctl -D $DB_DIR status

## Stop any existing PostgreSQL process

In [ ]:
!pg_ctl stop -D $DB_DIR -m fast || echo "Nothing to stop."

## Initialize the database if necessary

In [ ]:
import os

if not os.path.exists(DB_DIR) or not os.path.exists(os.path.join(DB_DIR, "PG_VERSION")):
    os.makedirs(DB_DIR, exist_ok=True)
    !initdb -D $DB_DIR

## Modify pg_hba.conf to allow trust authentication for user "notebook"

In [ ]:
with open(PG_HBA_PATH, "a") as pg_hba:
    pg_hba.write("\nlocal   all             notebook                                trust\n")
    pg_hba.write("host    all             notebook        127.0.0.1/32          trust\n")
    pg_hba.write("host    all             notebook        ::1/128               trust\n")

## Start PostgreSQL server

In [ ]:
import subprocess
import time
import atexit
import sys
import os

# Define paths
conda_prefix = sys.prefix
pg_ctl_path = os.path.join(conda_prefix, 'bin', 'pg_ctl')

def stop_postgres():
    subprocess.run([pg_ctl_path, "-D", DB_DIR, "stop"])

atexit.register(stop_postgres)

with open(LOGFILE, "a") as logfile:
    subprocess.Popen([pg_ctl_path, "-D", DB_DIR, "-l", LOGFILE, "start"],
                     stdout=logfile, stderr=logfile)

time.sleep(5)

## Create the "notebook" user with full permissions

In [ ]:
!psql -d postgres -c "CREATE USER notebook WITH SUPERUSER;" || echo "User already exists."
!psql -d postgres -c "ALTER USER notebook WITH PASSWORD NULL;"

# Connect

## Load & Configure SQLMagic

In [ ]:
%load_ext sql
%config SqlMagic.displaylimit = None

## Create Fresh sahuagin Database

In [ ]:
!psql -U notebook -d postgres -c "SELECT pg_terminate_backend(pid) FROM pg_stat_activity WHERE datname = 'sahuagin';"
!psql -U notebook -d postgres -c "DROP DATABASE IF EXISTS sahuagin;"
!psql -U notebook -d postgres -c "CREATE DATABASE sahuagin;"

## Connect to sahuagin Database

In [ ]:
%sql postgresql://notebook@localhost:5432/sahuagin

# Create Tables & Types

In [ ]:
%%sql

CREATE EXTENSION IF NOT EXISTS plpython3u;
CREATE EXTENSION IF NOT EXISTS citext;

-- Enum indicating supported programming languages for stored expressions.
DROP TYPE IF EXISTS prog_language CASCADE;
CREATE TYPE prog_language AS ENUM ('python');

-- Mechanism table replaces activable, variant, variable, and space.
-- A mechanism is defined by its programming language and serialized code.
-- It must have a unique (non-null) name.
DROP TABLE IF EXISTS mechanism CASCADE;
CREATE TABLE mechanism (
  id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
  name citext NOT NULL UNIQUE,
  pl prog_language NOT NULL,
  serialized text NOT NULL
);


-- Activation table now links mechanisms.
-- "from_mechanism" continues through an activation (by name) to the "to_mechanism",
-- and the activation is defined in the context of a "root_mechanism".
-- The unique constraint ensures that (from_mechanism, root_mechanism, name) is unique.
DROP TABLE IF EXISTS activation CASCADE;
CREATE TABLE activation (
  id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
  name citext NOT NULL,
  from_mechanism integer NOT NULL,
  root_mechanism integer NOT NULL,
  to_mechanism integer NOT NULL,
  CONSTRAINT uq_activation UNIQUE (from_mechanism, root_mechanism, name),
  CONSTRAINT fk_activation_from FOREIGN KEY (from_mechanism)
    REFERENCES mechanism(id) ON DELETE CASCADE,
  CONSTRAINT fk_activation_root FOREIGN KEY (root_mechanism)
    REFERENCES mechanism(id) ON DELETE CASCADE,
  CONSTRAINT fk_activation_to FOREIGN KEY (to_mechanism)
    REFERENCES mechanism(id) ON DELETE CASCADE
);

-- New table "unmasking"
-- This table provides a reference between a root mechanism, an activation,
-- and a mechanism that is being unmasked (unmasked_to_mechanism).
DROP TABLE IF EXISTS unmasking CASCADE;
CREATE TABLE unmasking (
  id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
  root_mechanism integer NOT NULL,
  activation integer NOT NULL,
  unmasked_to_mechanism integer NOT NULL,
  CONSTRAINT fk_unmasking_root FOREIGN KEY (root_mechanism)
    REFERENCES mechanism(id) ON DELETE CASCADE,
  CONSTRAINT fk_unmasking_activation FOREIGN KEY (activation)
    REFERENCES activation(id) ON DELETE CASCADE,
  CONSTRAINT fk_unmasking_unmasked FOREIGN KEY (unmasked_to_mechanism)
    REFERENCES mechanism(id) ON DELETE CASCADE
);

-- The entity table now associates an entity with a mechanism.
DROP TABLE IF EXISTS entity CASCADE;
CREATE TABLE entity (
  id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
  name citext NOT NULL,
  mechanism integer NOT NULL,
  CONSTRAINT fk_entity_mechanism FOREIGN KEY (mechanism)
    REFERENCES mechanism(id) ON DELETE CASCADE
);

-- Snapshot (state) of an entity at a moment in time.
DROP TABLE IF EXISTS observation CASCADE;
CREATE TABLE observation (
  id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
  entity integer NOT NULL,
  time double precision NOT NULL,
  CONSTRAINT uq_observation UNIQUE (entity, time),
  CONSTRAINT fk_observation_entity FOREIGN KEY (entity)
    REFERENCES entity(id) ON DELETE CASCADE
);

-- Locked relationships between observation and activation for partial re-generation.
DROP TABLE IF EXISTS locked_activation CASCADE;
CREATE TABLE locked_activation (
  id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
  observation integer NOT NULL,
  activation integer NOT NULL,
  CONSTRAINT fk_locked_activation_observation FOREIGN KEY (observation)
    REFERENCES observation(id) ON DELETE CASCADE,
  CONSTRAINT fk_locked_activation_activation FOREIGN KEY (activation)
    REFERENCES activation(id) ON DELETE CASCADE
);

-- Enum indicating the type of datum stored.
DROP TYPE IF EXISTS datum_type CASCADE;
CREATE TYPE datum_type AS ENUM ('string', 'number');

-- Abstract value representing mechanism states.
DROP TABLE IF EXISTS datum CASCADE;
CREATE TABLE datum (
  id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
  observation integer NOT NULL,
  activation integer NOT NULL,
  output_name citext NOT NULL,
  type datum_type NOT NULL,
  CONSTRAINT fk_datum_observation FOREIGN KEY (observation)
    REFERENCES observation(id) ON DELETE CASCADE,
  CONSTRAINT fk_datum_activation FOREIGN KEY (activation)
    REFERENCES activation(id) ON DELETE CASCADE
);

-- Numeric datum.
DROP TABLE IF EXISTS numeric_datum CASCADE;
CREATE TABLE numeric_datum (
  datum integer PRIMARY KEY,
  value double precision NOT NULL,
  CONSTRAINT fk_numeric_datum FOREIGN KEY (datum)
    REFERENCES datum(id) ON DELETE CASCADE
);

-- String datum.
DROP TABLE IF EXISTS string_datum CASCADE;
CREATE TABLE string_datum (
  datum integer PRIMARY KEY,
  value text NOT NULL,
  CONSTRAINT fk_string_datum FOREIGN KEY (datum)
    REFERENCES datum(id) ON DELETE CASCADE
);

-- Group of entity states.
DROP TABLE IF EXISTS sample CASCADE;
CREATE TABLE sample (
  id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
  name citext NOT NULL UNIQUE
);

-- Child sample relationship for sample pooling.
DROP TABLE IF EXISTS subsample CASCADE;
CREATE TABLE subsample (
  parent integer NOT NULL,
  child integer NOT NULL,
  PRIMARY KEY (parent, child),
  CONSTRAINT fk_subsample_parent FOREIGN KEY (parent)
    REFERENCES sample(id) ON DELETE CASCADE,
  CONSTRAINT fk_subsample_child FOREIGN KEY (child)
    REFERENCES sample(id) ON DELETE CASCADE
);

-- Entity observed in a sample.
DROP TABLE IF EXISTS sample_entity CASCADE;
CREATE TABLE sample_entity (
  entity integer PRIMARY KEY,
  sample integer NOT NULL,
  CONSTRAINT fk_sample_entity_entity FOREIGN KEY (entity)
    REFERENCES entity(id) ON DELETE CASCADE,
  CONSTRAINT fk_sample_entity_sample FOREIGN KEY (sample)
    REFERENCES sample(id) ON DELETE CASCADE
);


# Triggers

## Delete Orphan possible_attr

In [ ]:
%%sql

CREATE OR REPLACE FUNCTION check_delete_possible_attr_from_effect()
RETURNS TRIGGER AS $$
DECLARE
    orphan_count INTEGER;
BEGIN
    -- Check the to_modify_possible_attr_id column
    IF OLD.to_modify_possible_attr_id IS NOT NULL THEN
        SELECT
            (SELECT COUNT(*) FROM effect WHERE to_modify_possible_attr_id = OLD.to_modify_possible_attr_id)
          + (SELECT COUNT(*) FROM effect WHERE activating_possible_attr_id = OLD.to_modify_possible_attr_id)
          + (SELECT COUNT(*) FROM attr_value WHERE possible_attr_id = OLD.to_modify_possible_attr_id)
          INTO orphan_count;
          
        IF orphan_count = 0 THEN
            DELETE FROM possible_attr WHERE id = OLD.to_modify_possible_attr_id;
        END IF;
    END IF;
    
    -- Check the activating_possible_attr_id column (if different)
    IF OLD.activating_possible_attr_id IS NOT NULL
       AND (OLD.activating_possible_attr_id <> OLD.to_modify_possible_attr_id) THEN
        SELECT
            (SELECT COUNT(*) FROM effect WHERE to_modify_possible_attr_id = OLD.activating_possible_attr_id)
          + (SELECT COUNT(*) FROM effect WHERE activating_possible_attr_id = OLD.activating_possible_attr_id)
          + (SELECT COUNT(*) FROM attr_value WHERE possible_attr_id = OLD.activating_possible_attr_id)
          INTO orphan_count;
          
        IF orphan_count = 0 THEN
            DELETE FROM possible_attr WHERE id = OLD.activating_possible_attr_id;
        END IF;
    END IF;
    
    RETURN NULL;
END;
$$ LANGUAGE plpgsql;

CREATE OR REPLACE FUNCTION check_delete_possible_attr_from_attr_val()
RETURNS TRIGGER AS $$
DECLARE
    orphan_count INTEGER;
BEGIN
    SELECT
        (SELECT COUNT(*) FROM effect 
           WHERE to_modify_possible_attr_id = OLD.possible_attr_id
              OR activating_possible_attr_id = OLD.possible_attr_id)
      + (SELECT COUNT(*) FROM attr_value WHERE possible_attr_id = OLD.possible_attr_id)
      INTO orphan_count;
      
    IF orphan_count = 0 THEN
        DELETE FROM possible_attr WHERE id = OLD.possible_attr_id;
    END IF;
    
    RETURN NULL;
END;
$$ LANGUAGE plpgsql;

CREATE OR REPLACE TRIGGER trg_check_delete_possible_attr_from_effect
AFTER DELETE ON effect
FOR EACH ROW
EXECUTE FUNCTION check_delete_possible_attr_from_effect();

CREATE OR REPLACE TRIGGER trg_check_delete_possible_attr_from_attr_val
AFTER DELETE ON attr_value
FOR EACH ROW
EXECUTE FUNCTION check_delete_possible_attr_from_attr_val();

## Delete Orphan Span

In [ ]:
%%sql

CREATE OR REPLACE FUNCTION delete_orphaned_span() RETURNS TRIGGER AS $$
BEGIN
    DELETE FROM span WHERE id = OLD.span_id;
    RETURN OLD;
END;
$$ LANGUAGE plpgsql;

CREATE OR REPLACE TRIGGER trg_delete_span_after_activation_delete
AFTER DELETE ON add_span_eft
FOR EACH ROW
EXECUTE FUNCTION delete_orphaned_span();

# Create Indexes

# Debug

## Create Log Table

In [ ]:
%%sql

DROP TABLE IF EXISTS debug_log CASCADE;
CREATE TABLE debug_log (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    log_time timestamp DEFAULT CURRENT_TIMESTAMP,
    procedure_name varchar(255),
    log_message text
);

## Create Log Function

In [ ]:
%%sql

CREATE OR REPLACE FUNCTION debug_log(
    p_procedure_name varchar,
    p_log_message text
) RETURNS void
LANGUAGE plpgsql
AS $$
BEGIN
    INSERT INTO debug_log (procedure_name, log_message, log_time)
    VALUES (p_procedure_name, p_log_message, now());
END;
$$;

## Clear Log Table

In [ ]:
%%sql

TRUNCATE TABLE debug_log RESTART IDENTITY CASCADE;

In [ ]:
%%sql

SELECT * FROM debug_log;

# New

In [ ]:
DROP FUNCTION IF EXISTS get_activation_full_path(integer);

CREATE OR REPLACE FUNCTION get_activation_full_path(activation_id integer)
RETURNS text
LANGUAGE plpgsql
STABLE
AS $$
DECLARE
    full_path text;
BEGIN
    WITH RECURSIVE act_path AS (
        -- Start with the given activation.
        SELECT 
            id,
            name,
            from_mechanism,
            root_mechanism,
            to_mechanism,
            name AS full_path
        FROM activation
        WHERE id = activation_id

        UNION ALL

        -- Recursively join parent's activation where parent's to_mechanism
        -- matches the child's from_mechanism and they share the same root.
        SELECT 
            p.id,
            p.name,
            p.from_mechanism,
            p.root_mechanism,
            p.to_mechanism,
            p.name || '/' || ap.full_path AS full_path
        FROM activation p
        JOIN act_path ap 
          ON p.to_mechanism = ap.from_mechanism 
         AND p.root_mechanism = ap.root_mechanism
    )
    -- The root activation in the chain will have no parent – i.e. no activation
    -- exists such that its to_mechanism equals this activation's from_mechanism.
    SELECT full_path INTO full_path
    FROM act_path
    WHERE NOT EXISTS (
        SELECT 1 
        FROM activation p2
        WHERE p2.to_mechanism = act_path.from_mechanism 
          AND p2.root_mechanism = act_path.root_mechanism
    )
    LIMIT 1;
    
    RETURN full_path;
END;
$$;

In [ ]:
CREATE OR REPLACE PROCEDURE generate_entity_state(
    p_entity_name VARCHAR,
    p_time        DOUBLE PRECISION
)
LANGUAGE plpython3u
AS $$

from typing import Optional, Any, Dict, Tuple

# Global dictionaries to record outputs, activation paths, and unmasking records.
act_outputs: Dict[str, Any] = {}        # Maps resolved output paths to output values.
act_paths: Dict[str, int] = {}            # Maps resolved activation paths to activation IDs.
locked_act_paths: Dict[str, int] = {}     # Maps full activation paths for locked activations to activation IDs.
# unmasking_cache maps effective activation paths to a tuple (depth, unmasked_to_mechanism)
unmasking_cache: Dict[str, Tuple[int, int]] = {}

# Global state variables.
current_activation_id: Optional[int] = None  # Current mechanism activation ID.
current_activation_path: str = ""              # Slash-separated path of activations.
is_regeneration: bool = False                  # True if an observation already exists.

def resolve_path(input_path: str) -> str:
    """
    Resolves an input path string according to these rules:
      - Absolute: if input starts with '/', returns the path without the leading slash.
      - Relative: if it starts with './' or '../', resolves relative to the current activation.
      - Otherwise, treats as relative to the current activation.
    """
    global current_activation_path
    if input_path.startswith("/"):
        return input_path.lstrip("/")
    elif input_path.startswith("./") or input_path.startswith(".."):
        base_components = current_activation_path.split("/") if current_activation_path else []
        resolved_components = base_components.copy()
        for part in input_path.split("/"):
            if part in (".", ""):
                continue
            elif part == "..":
                if resolved_components:
                    resolved_components.pop()
                else:
                    plpy.error("Path resolution error: cannot go above the root activation")
            else:
                resolved_components.append(part)
        return "/".join(resolved_components)
    else:
        return f"{current_activation_path}/{input_path}" if current_activation_path else input_path

def load_locked_activations(observation_id: int) -> None:
    """
    Loads all locked activations (and their outputs) for the given observation.
    Assumes the existence of a helper SQL function get_activation_full_path(activation_id)
    that returns the full (slash-separated) activation path.
    """
    global act_paths, act_outputs, locked_act_paths
    sql_locked = """
        SELECT la.activation, get_activation_full_path(la.activation) AS full_path
        FROM locked_activation la
        WHERE la.observation = $1
    """
    res_locked = plpy.execute(sql_locked, [observation_id])
    for row in res_locked:
        full_path = row["full_path"]
        activation_id = row["activation"]
        locked_act_paths[full_path] = activation_id
        act_paths[full_path] = activation_id

        # Load outputs produced by this locked activation.
        sql_datum = (
            "SELECT d.id, d.output_name, d.type, "
            "CASE WHEN d.type = 'number' THEN "
            "    (SELECT value FROM numeric_datum WHERE datum = d.id) "
            "ELSE "
            "    (SELECT value FROM string_datum WHERE datum = d.id) END AS value "
            "FROM datum d "
            "WHERE d.observation = $1 AND d.activation = $2"
        )
        res_datum = plpy.execute(sql_datum, [observation_id, activation_id])
        for d in res_datum:
            full_output_path = f"{full_path}/{d['output_name']}"
            act_outputs[full_output_path] = d["value"]

def run_mechanism(mech_id: int,
                  activation_name: Optional[str],
                  activation_id: Optional[int],
                  root_mech_id: int,
                  observation_id: int) -> None:
    """
    Looks up the mechanism by id, injects helper functions into its namespace,
    and executes its serialized code.
    
    In regeneration mode, if the current activation (by full path) is pre-loaded
    as locked, its code is not re-executed. For non-locked activations, any old
    datum rows are deleted so that new ones will replace them.
    
    Additionally, before executing the mechanism's code, the function checks the
    unmasking cache. If an unmasking record applies at the current activation's path,
    the originally intended activation is entirely replaced with the unmasked mechanism.
    """
    sql = "SELECT id, name, pl, serialized FROM mechanism WHERE id = $1"
    res = plpy.execute(sql, [mech_id])
    if res.nrows() == 0:
        plpy.error("Mechanism with id %s not found" % mech_id)
    mech = res[0]

    global current_activation_id, current_activation_path, act_paths, is_regeneration, locked_act_paths, unmasking_cache
    prev_activation_id = current_activation_id
    prev_activation_path = current_activation_path

    try:
        # Update the activation path if a local activation name is provided.
        if activation_name:
            current_activation_path = (f"{current_activation_path}/{activation_name}"
                                       if current_activation_path else activation_name)

        # Unmasking check: if the current activation path matches an entry in the unmasking cache,
        # then replace the intended mechanism with the unmasked mechanism.
        if current_activation_path in unmasking_cache:
            _, unmasked_to_mech = unmasking_cache[current_activation_path]
            run_mechanism(unmasked_to_mech, None, None, root_mech_id, observation_id)
            return

        # In regeneration, if this activation was locked, skip re-execution.
        if is_regeneration and current_activation_path in locked_act_paths:
            return

        # For non-locked activations in regeneration, delete any old datum rows and clear their outputs.
        if is_regeneration and activation_id is not None:
            plpy.execute("DELETE FROM datum WHERE observation = $1 AND activation = $2", [observation_id, activation_id])
            keys_to_delete = [k for k in act_outputs if k.startswith(current_activation_path + "/")]
            for k in keys_to_delete:
                del act_outputs[k]

        # Record the new activation id if provided.
        if activation_id is not None:
            current_activation_id = activation_id
            act_paths[current_activation_path] = activation_id

        # Helper function to fetch an input value from previously recorded outputs.
        def use_input(path: str) -> Any:
            resolved = resolve_path(path)
            if resolved in act_outputs:
                return act_outputs[resolved]
            else:
                plpy.error("No output found for resolved path: " + resolved)
        
        # Helper function to add a new output.
        def add_output(name: str, value: Any) -> Any:
            full_output_path = resolve_path(name)
            if full_output_path in act_outputs:
                plpy.error("Output with name '%s' already exists in the current activation" % name)
            
            datum_type_val = "number" if isinstance(value, (int, float)) else "string"
            sql_ins = (
                "INSERT INTO datum(observation, activation, output_name, type) "
                "VALUES ($1, $2, $3, $4) RETURNING id"
            )
            activation_val = current_activation_id if current_activation_id is not None else None
            res_ins = plpy.execute(sql_ins, [observation_id, activation_val, name, datum_type_val])
            datum_id = res_ins[0]["id"]
            
            if datum_type_val == "number":
                plpy.execute("INSERT INTO numeric_datum(datum, value) VALUES ($1, $2)", [datum_id, value])
            else:
                plpy.execute("INSERT INTO string_datum(datum, value) VALUES ($1, $2)", [datum_id, str(value)])
            
            act_outputs[full_output_path] = value
            return value
        
        # Helper function to activate a child mechanism.
        def activate(mechanism_name: str, local_activation_name: Optional[str] = None) -> int:
            if local_activation_name is None:
                local_activation_name = mechanism_name
            full_activation_path = resolve_path(local_activation_name)
            if full_activation_path in act_paths:
                plpy.error("Activation with name '%s' already exists in the current activation" % local_activation_name)
            
            sql_lookup = "SELECT id FROM mechanism WHERE name = $1 LIMIT 1"
            res_lookup = plpy.execute(sql_lookup, [mechanism_name])
            if res_lookup.nrows() == 0:
                plpy.error("Mechanism with name %s not found" % mechanism_name)
            new_mech_id = res_lookup[0]["id"]

            # Retrieve and cache unmasking records for the child mechanism.
            sql_unmask = "SELECT activation, unmasked_to_mechanism FROM unmasking WHERE root_mechanism = $1"
            res_unmask = plpy.execute(sql_unmask, [new_mech_id])
            for row in res_unmask:
                res_rel = plpy.execute("SELECT get_activation_full_path($1) AS rel_path", [row["activation"]])
                if res_rel.nrows() > 0:
                    rel_path = res_rel[0]["rel_path"]
                    effective_key = f"{current_activation_path}/{rel_path}" if current_activation_path else rel_path
                    depth = len(rel_path.split("/"))
                    if effective_key in unmasking_cache:
                        existing_depth, _ = unmasking_cache[effective_key]
                        if depth > existing_depth:
                            unmasking_cache[effective_key] = (depth, row["unmasked_to_mechanism"])
                    else:
                        unmasking_cache[effective_key] = (depth, row["unmasked_to_mechanism"])
            
            sql_act = (
                "INSERT INTO activation(name, from_mechanism, root_mechanism, to_mechanism) "
                "VALUES ($1, $2, $3, $4) RETURNING id"
            )
            res_act = plpy.execute(sql_act, [local_activation_name, mech_id, root_mech_id, new_mech_id])
            new_activation_id = res_act[0]["id"]
            run_mechanism(new_mech_id, local_activation_name, new_activation_id, root_mech_id, observation_id)
            return new_activation_id
        
        # Helper function to reject (delete) an activation and its descendant activations.
        def reject(local_activation_name: str) -> None:
            full_path = resolve_path(local_activation_name)
            if full_path not in act_paths:
                plpy.error("Activation with resolved path %s not found" % full_path)
            act_id = act_paths[full_path]
            sql_reject = (
                "WITH RECURSIVE act_tree AS ("
                "  SELECT id FROM activation WHERE id = $1 "
                "  UNION ALL "
                "  SELECT a.id FROM activation a JOIN act_tree at ON a.from_mechanism = at.id"
                ") DELETE FROM activation WHERE id IN (SELECT id FROM act_tree)"
            )
            plpy.execute(sql_reject, [act_id])
            del act_paths[full_path]
        
        # Prepare the local namespace with the helper functions.
        local_ns = {
            "use_input": use_input,
            "add_output": add_output,
            "activate": activate,
            "reject": reject,
        }
        
        # Execute the serialized mechanism code.
        exec(mech["serialized"], local_ns)
        if "main" not in local_ns:
            plpy.error("Mechanism code does not define a main() function")
        local_ns["main"]()
    
    finally:
        # Restore the previous activation context.
        current_activation_id = prev_activation_id
        current_activation_path = prev_activation_path

# === Main body of generate_entity_state ===

# Lookup the entity record.
sql_entity = "SELECT id, mechanism FROM entity WHERE name = $1 LIMIT 1"
res_entity = plpy.execute(sql_entity, [p_entity_name])
if res_entity.nrows() == 0:
    plpy.error("Entity with name '%s' not found" % p_entity_name)
entity_rec = res_entity[0]
entity_id = entity_rec["id"]
root_mech_id = entity_rec["mechanism"]

# Find or create an observation.
sql_obs = "SELECT id FROM observation WHERE entity = $1 AND time = $2 LIMIT 1"
res_obs = plpy.execute(sql_obs, [entity_id, p_time])
if res_obs.nrows() == 0:
    res_insert = plpy.execute(
        "INSERT INTO observation(entity, time) VALUES ($1, $2) RETURNING id",
        [entity_id, p_time]
    )
    observation_id = res_insert[0]["id"]
else:
    observation_id = res_obs[0]["id"]
    is_regeneration = True
    load_locked_activations(observation_id)

# Start by running the root mechanism.
run_mechanism(root_mech_id, None, None, root_mech_id, observation_id)

$$;

# Create Functions & Procedures

## Set Span as Subvariant

In [ ]:
%%sql

CREATE OR REPLACE PROCEDURE set_span_as_subvariant(
    p_attribute_name VARCHAR,
    p_span_label VARCHAR,
    p_variant_name VARCHAR
)
LANGUAGE plpgsql
AS $$
DECLARE
    v_attribute_id INTEGER;
    v_span_id INTEGER;
    v_variant_id INTEGER;
BEGIN
    -- Find the attribute by name.
    SELECT id
      INTO v_attribute_id
      FROM attribute
     WHERE name = p_attribute_name;
    IF NOT FOUND THEN
        RAISE EXCEPTION 'Attribute "%" not found.', p_attribute_name;
    END IF;

    -- Find the span by attribute and label.
    SELECT id
      INTO v_span_id
      FROM span
     WHERE attribute_id = v_attribute_id
       AND label = p_span_label;
    IF NOT FOUND THEN
        RAISE EXCEPTION 'Span with label "%" for attribute "%" not found.', p_span_label, p_attribute_name;
    END IF;

    -- Find or create the variant by name.
    SELECT id
      INTO v_variant_id
      FROM variant
     WHERE name = p_variant_name;
    IF NOT FOUND THEN
        INSERT INTO variant (name)
             VALUES (p_variant_name)
         RETURNING id INTO v_variant_id;
    END IF;

    -- Insert the association into subvariant_span.
    INSERT INTO subvariant_span (span_id, variant_id)
         VALUES (v_span_id, v_variant_id)
    ON CONFLICT DO NOTHING;

    RAISE NOTICE 'Subvariant span set for attribute %, span %, variant %.',
                 p_attribute_name, p_span_label, p_variant_name;
END;
$$;

## Unset Span as Subvariant

In [ ]:
%%sql

CREATE OR REPLACE PROCEDURE unset_span_as_subvariant(
    p_attribute_name VARCHAR,
    p_span_label VARCHAR,
    p_variant_name VARCHAR
)
LANGUAGE plpgsql
AS $$
DECLARE
    v_attribute_id INTEGER;
    v_span_id INTEGER;
    v_variant_id INTEGER;
    v_deleted_count INTEGER;
BEGIN
    -- Find the attribute by name.
    SELECT id
      INTO v_attribute_id
      FROM attribute
     WHERE name = p_attribute_name;
    IF NOT FOUND THEN
        RAISE EXCEPTION 'Attribute "%" not found.', p_attribute_name;
    END IF;

    -- Find the span by attribute and label.
    SELECT id
      INTO v_span_id
      FROM span
     WHERE attribute_id = v_attribute_id
       AND label = p_span_label;
    IF NOT FOUND THEN
        RAISE EXCEPTION 'Span with label "%" for attribute "%" not found.', p_span_label, p_attribute_name;
    END IF;

    -- Find the variant by name.
    SELECT id
      INTO v_variant_id
      FROM variant
     WHERE name = p_variant_name;
    IF NOT FOUND THEN
        RAISE EXCEPTION 'Variant "%" not found.', p_variant_name;
    END IF;

    -- Delete the association from subvariant_span.
    DELETE FROM subvariant_span
     WHERE span_id = v_span_id
       AND variant_id = v_variant_id;
    
    GET DIAGNOSTICS v_deleted_count = ROW_COUNT;
    
    IF v_deleted_count = 0 THEN
        RAISE NOTICE 'No subvariant span association existed for attribute %, span %, variant %.',
                     p_attribute_name, p_span_label, p_variant_name;
    ELSE
        RAISE NOTICE 'Subvariant span association removed for attribute %, span %, variant %.',
                     p_attribute_name, p_span_label, p_variant_name;
    END IF;
END;
$$;

## Discrete Attribute

### Add

In [ ]:
%%sql

CREATE OR REPLACE PROCEDURE add_discrete_attribute(
    in_name       VARCHAR(255),
    in_spans      JSON  -- JSON array: either [["span1", 10], ["span2", 20, "subvariantX"]] or ["span1", "span2"]
)
LANGUAGE plpgsql
AS $$
DECLARE
    new_attr_id       INTEGER;
    span_count        INTEGER;
    span_label        VARCHAR(255);
    span_weight       DOUBLE PRECISION;
    normalized_weight DOUBLE PRECISION;
    total_weight      DOUBLE PRECISION := 0;
    first_elem_type   TEXT;
    spans_json        JSON;
    i                 INTEGER;
BEGIN
    -- Create the new attribute.
    INSERT INTO attribute (name, type)
      VALUES (in_name, 'discrete')
      RETURNING id INTO new_attr_id;

    IF in_spans IS NOT NULL THEN
        spans_json := in_spans::json;
        span_count := json_array_length(spans_json);

        IF span_count > 0 THEN
            first_elem_type := json_typeof(spans_json->0);
        ELSE
            first_elem_type := '';
        END IF;

        IF first_elem_type = 'array' THEN
            -- First pass: compute the total weight.
            FOR i IN 0..span_count - 1 LOOP
                total_weight := total_weight + (((spans_json->i)->>1)::DOUBLE PRECISION);
            END LOOP;

            -- Second pass: insert spans with normalized weight.
            FOR i IN 0..span_count - 1 LOOP
                span_label := (spans_json->i)->>0;
                span_weight := ((spans_json->i)->>1)::DOUBLE PRECISION;
                normalized_weight := span_weight / total_weight;
                INSERT INTO span (attribute_id, label, type, is_pinned, weight)
                  VALUES (new_attr_id, span_label, 'discrete', false, normalized_weight);
                
                -- If a third element exists, call set_subvariant_span.
                IF json_array_length(spans_json->i) > 2 THEN
                    CALL set_span_as_subvariant(in_name, span_label, (spans_json->i)->>2);
                END IF;
            END LOOP;
        ELSE
            -- For a JSON array of labels only, call the helper procedure for each span.
            FOR i IN 0..span_count - 1 LOOP
                span_label := spans_json->>i;
                CALL add_discrete_span(new_attr_id, span_label);
            END LOOP;
        END IF;
    END IF;
END;
$$;

### Get

In [ ]:
%%sql

CREATE OR REPLACE FUNCTION get_discrete_attribute(p_attribute_name varchar)
RETURNS TABLE (
    attribute   varchar,
    type        attr_type,
    span_label  varchar,
    span_weight double precision,
    is_pinned      boolean
)
LANGUAGE plpgsql
AS
$$
BEGIN
  RETURN QUERY
    SELECT 
      a.name,
      a.type,
      s.label,
      s.weight,
      s.is_pinned
    FROM attribute a
    JOIN span s ON s.attribute_id = a.id
    WHERE a.name = p_attribute_name
      AND NOT EXISTS (
          SELECT 1 
          FROM add_span_eft ase 
          WHERE ase.span_id = s.id
      );
END;
$$;

### Delete

In [ ]:
%%sql

CREATE OR REPLACE PROCEDURE delete_discrete_attribute(p_attribute_name varchar)
LANGUAGE plpgsql
AS
$$
BEGIN
  DELETE FROM attribute
  WHERE name = p_attribute_name
    AND type = 'discrete';
END;
$$;

## Redistribute Unpinned Spans

In [ ]:
%%sql

CREATE OR REPLACE FUNCTION redistribute_unpinned_spans(
    p_attribute_id INTEGER,
    p_modified_span_id INTEGER,
    p_new_weight DOUBLE PRECISION,
    p_old_weight DOUBLE PRECISION
) RETURNS VOID
LANGUAGE plpgsql
AS $$
DECLARE
    v_pinned_weight DOUBLE PRECISION;
    v_target         DOUBLE PRECISION;
BEGIN
    -- Compute the total weight of pinned discrete spans.
    SELECT COALESCE(SUM(weight), 0)
      INTO v_pinned_weight
      FROM span
     WHERE attribute_id = p_attribute_id
       AND type = 'discrete'
       AND is_pinned = true;
       
    -- The available weight for all unpinned spans.
    v_target := 1.0 - v_pinned_weight;
    
    -- If there’s only one unpinned span (v_target equals p_old_weight), then
    -- the new weight must equal the entire target.
    IF (v_target - p_old_weight) = 0 THEN
       IF p_new_weight <> v_target THEN
           RAISE EXCEPTION 'Cannot change weight; only one unpinned span exists and its weight must be %', v_target;
       END IF;
    END IF;
    
    -- Update all unpinned discrete spans:
    -- • The modified span gets the new weight.
    -- • All others are scaled proportionally.
    UPDATE span
       SET weight = CASE 
                      WHEN id = p_modified_span_id THEN p_new_weight
                      ELSE weight * ((v_target - p_new_weight) / (v_target - p_old_weight))
                    END
     WHERE attribute_id = p_attribute_id
       AND type = 'discrete'
       AND is_pinned = false;
END;
$$;

## Add Discrete Span to Attribute

In [ ]:
%%sql

CREATE OR REPLACE PROCEDURE add_disc_span_to_attr(
    p_attribute_id INTEGER,
    p_label VARCHAR(255)
)
LANGUAGE plpgsql
AS $$
DECLARE
    new_span_id     INTEGER;
    v_total_weight  DOUBLE PRECISION;
    v_pinned_weight DOUBLE PRECISION;
    v_target        DOUBLE PRECISION;
    v_count_old     INTEGER;
    candidate       DOUBLE PRECISION;
BEGIN
    -- Compute the total weight for all discrete spans for this attribute.
    SELECT COALESCE(SUM(weight), 0)
      INTO v_total_weight
      FROM span
     WHERE attribute_id = p_attribute_id
       AND type = 'discrete';

    -- If no spans exist, simply insert the first span with weight 1.
    IF v_total_weight = 0 THEN
        INSERT INTO span(attribute_id, label, type, is_pinned, weight)
        VALUES (p_attribute_id, p_label, 'discrete', false, 1.0);
        RETURN;
    END IF;
    
    -- Compute the total weight of pinned discrete spans.
    SELECT COALESCE(SUM(weight), 0)
      INTO v_pinned_weight
      FROM span
     WHERE attribute_id = p_attribute_id
       AND type = 'discrete'
       AND is_pinned = true;
    
    -- Calculate the available target weight for all unpinned spans.
    v_target := 1.0 - v_pinned_weight;
    IF v_target <= 0 THEN
        RAISE EXCEPTION 'No available weight for unpinned spans (v_target = %)', v_target;
    END IF;
    
    -- Count the existing non-pinned discrete spans.
    SELECT COUNT(*)
      INTO v_count_old
      FROM span
     WHERE attribute_id = p_attribute_id
       AND type = 'discrete'
       AND is_pinned = false;
    
    -- Insert the new unpinned span with a temporary weight of 0.
    INSERT INTO span(attribute_id, label, type, is_pinned, weight)
    VALUES (p_attribute_id, p_label, 'discrete', false, 0.0)
    RETURNING id INTO new_span_id;
    
    -- Compute the candidate weight for the new span.
    candidate := v_target / (v_count_old + 1);
    
    -- Redistribute the weights among all unpinned spans
    PERFORM redistribute_unpinned_spans(p_attribute_id, new_span_id, candidate, 0);
END;
$$;

## Modify Discrete Span Weight

In [ ]:
%%sql

CREATE OR REPLACE PROCEDURE modify_disc_span_weight(
    p_span_id    INTEGER,
    p_new_weight DOUBLE PRECISION
)
LANGUAGE plpgsql
AS $$
DECLARE
    v_attribute_id INTEGER;
    v_current_weight DOUBLE PRECISION;
    v_is_pinned    BOOLEAN;
    v_type         TEXT;
    v_pinned_weight DOUBLE PRECISION;
    v_target        DOUBLE PRECISION;
    v_other_sum     DOUBLE PRECISION;
BEGIN
    -- Get the current span's details.
    SELECT attribute_id, weight, is_pinned, type
      INTO v_attribute_id, v_current_weight, v_is_pinned, v_type
      FROM span
     WHERE id = p_span_id;
     
    -- Ensure the span exists, is of type 'discrete' and is not pinned.
    IF NOT FOUND OR v_type <> 'discrete' OR v_is_pinned THEN
       RAISE EXCEPTION 'Span % not found or not a modifiable discrete span', p_span_id;
    END IF;
    
    -- Compute the total pinned weight for this attribute.
    SELECT COALESCE(SUM(weight), 0)
      INTO v_pinned_weight
      FROM span
     WHERE attribute_id = v_attribute_id
       AND type = 'discrete'
       AND is_pinned = true;
       
    v_target := 1.0 - v_pinned_weight;
    
    -- Validate the new weight is between 0 and the available target.
    IF p_new_weight < 0 OR p_new_weight > v_target THEN
       RAISE EXCEPTION 'Invalid new weight: % (must be between 0 and %)', p_new_weight, v_target;
    END IF;
    
    -- Compute the total weight for all other unpinned spans.
    SELECT COALESCE(SUM(weight), 0)
      INTO v_other_sum
      FROM span
     WHERE attribute_id = v_attribute_id
       AND type = 'discrete'
       AND is_pinned = false
       AND id <> p_span_id;
       
    IF v_other_sum = 0 THEN
       -- There is only one unpinned span. Its weight must equal the available target.
       IF p_new_weight <> v_target THEN
         RAISE EXCEPTION 'Only one unpinned span exists. Its weight must be %', v_target;
       ELSE
         UPDATE span SET weight = p_new_weight WHERE id = p_span_id;
       END IF;
    ELSE
       -- Use the helper function to redistribute weights.
       PERFORM redistribute_unpinned_spans(v_attribute_id, p_span_id, p_new_weight, v_current_weight);
    END IF;
END;
$$;

## Continuous Attribute

### Add

In [ ]:
%%sql

CREATE OR REPLACE PROCEDURE add_continuous_attribute(
    p_name           VARCHAR(255),
    p_min_value      DOUBLE PRECISION,
    p_mode_value     DOUBLE PRECISION,
    p_max_value      DOUBLE PRECISION,
    p_concentration  DOUBLE PRECISION,
    p_skew           DOUBLE PRECISION,
    p_decimals       INTEGER,
    p_units          VARCHAR(255),
    p_spans          JSON DEFAULT NULL  -- JSON array shortcut for continuous spans; tuples: [label, max_boundary, (optional subvariant)]
)
LANGUAGE plpgsql
AS $$
DECLARE
    v_attr_id   INTEGER;
    span_count  INTEGER;
    i           INTEGER;
    current_min DOUBLE PRECISION;
    new_max     DOUBLE PRECISION;
    span_label  VARCHAR(255);
    spans_json  JSON;
BEGIN
    -- Insert the continuous attribute record.
    INSERT INTO attribute
      (name, type, decimals, max_value, min_value,
       mode_value, concentration, skew, units)
    VALUES
      (p_name, 'continuous', p_decimals, p_max_value,
       p_min_value, p_mode_value, p_concentration, p_skew, p_units)
    RETURNING id INTO v_attr_id;
    
    IF p_spans IS NOT NULL THEN
        spans_json := p_spans::json;
        span_count := json_array_length(spans_json);
        current_min := p_min_value;
        
        FOR i IN 0..span_count - 1 LOOP
            -- Each JSON tuple is assumed to be [label, new_max, (optional subvariant)].
            span_label := (spans_json->i)->>0;
            new_max := ((spans_json->i)->>1)::DOUBLE PRECISION;
            
            -- Validate that the new maximum is in the valid interval.
            IF new_max <= current_min OR new_max > p_max_value THEN
                RAISE EXCEPTION 'Invalid span range: new_max (%) is not in the valid interval (% - %)',
                    new_max, current_min, p_max_value;
            END IF;
            
            -- Insert the continuous span.
            INSERT INTO span(attribute_id, label, type, min_value, max_value)
            VALUES(v_attr_id, span_label, 'continuous', current_min, new_max);
            
            -- If a third element exists, call set_subvariant_span.
            IF json_array_length(spans_json->i) > 2 THEN
                CALL set_span_as_subvariant(p_name, span_label, (spans_json->i)->>2);
            END IF;
            
            current_min := new_max;
        END LOOP;
        
        -- If the JSON array does not reach the attribute's overall max, raise an exception.
        IF current_min < p_max_value THEN
            RAISE EXCEPTION 'Incomplete continuous span definitions: final span max is % which is less than the attribute maximum %', current_min, p_max_value;
        END IF;
    END IF;
END;
$$;

### Get

In [ ]:
%%sql

CREATE OR REPLACE FUNCTION get_continuous_attribute(p_attribute_name varchar)
RETURNS TABLE (
    name           varchar,
    type           attr_type,
    min            double precision,
    mode           double precision,
    max            double precision,
    concentration  double precision,
    skew           double precision,
    units          varchar,
    decimals       integer,
    span_label     varchar,
    span_min       double precision,
    span_max       double precision
)
LANGUAGE plpgsql
AS
$$
BEGIN
  RETURN QUERY
    SELECT 
      a.name,
      a.type,
      a.min_value,
      a.mode_value,
      a.max_value,
      a.concentration,
      a.skew,
      a.units,
      a.decimals,
      s.label,
      s.min_value,
      s.max_value
    FROM attribute a
    LEFT JOIN span s ON s.attribute_id = a.id
    WHERE a.name = p_attribute_name
      AND a.type = 'continuous';
END;
$$;

### Delete

In [ ]:
%%sql

CREATE OR REPLACE PROCEDURE delete_continuous_attribute(p_attribute_name varchar)
LANGUAGE plpgsql
AS
$$
BEGIN
  DELETE FROM attribute
  WHERE name = p_attribute_name
    AND type = 'continuous';
END;
$$;

## Add Continuous Span to Attribute

In [ ]:
%%sql

CREATE OR REPLACE PROCEDURE add_cont_span_to_attr(
    p_attribute_id integer,
    p_label varchar(255),
    p_new_min double precision,
    p_new_max double precision
)
LANGUAGE plpgsql
AS $$
DECLARE
    v_attr_min double precision;
    v_attr_max double precision;
    v_count integer;
    v_first_id integer;
    v_first_min double precision;
    v_first_max double precision;
    v_last_id integer;
    v_last_min double precision;
    v_last_max double precision;
BEGIN
    -- Get attribute’s overall range (continuous only)
    SELECT min_value, max_value
      INTO v_attr_min, v_attr_max
      FROM attribute
     WHERE id = p_attribute_id
       AND type = 'continuous'
     LIMIT 1;
     
    IF v_attr_min IS NULL THEN
        RAISE EXCEPTION 'Attribute not found or not continuous';
    END IF;
    
    -- Validate requested span
    IF p_new_min < v_attr_min OR p_new_max > v_attr_max OR p_new_min >= p_new_max THEN
        RAISE EXCEPTION 'Invalid span range';
    END IF;
    
    -- If no continuous spans exist, insert one covering the full range
    SELECT COUNT(*) INTO v_count
      FROM span
     WHERE attribute_id = p_attribute_id
       AND type = 'continuous';
       
    IF v_count = 0 THEN
        INSERT INTO span(attribute_id, label, type, min_value, max_value)
        VALUES(p_attribute_id, p_label, 'continuous', v_attr_min, v_attr_max);
        RETURN;
    END IF;
    
    -- Adjust overlapping spans: first overlapping span (ordered by min_value)
    SELECT id, min_value, max_value
      INTO v_first_id, v_first_min, v_first_max
      FROM span
     WHERE attribute_id = p_attribute_id
       AND type = 'continuous'
       AND max_value > p_new_min
       AND min_value < p_new_max
     ORDER BY min_value ASC
     LIMIT 1;
     
    IF v_first_min < p_new_min THEN
        UPDATE span SET max_value = p_new_min WHERE id = v_first_id;
    END IF;
    
    -- Adjust overlapping spans: last overlapping span (ordered by max_value)
    SELECT id, min_value, max_value
      INTO v_last_id, v_last_min, v_last_max
      FROM span
     WHERE attribute_id = p_attribute_id
       AND type = 'continuous'
       AND max_value > p_new_min
       AND min_value < p_new_max
     ORDER BY max_value DESC
     LIMIT 1;
     
    IF v_last_max > p_new_max THEN
        UPDATE span SET min_value = p_new_max WHERE id = v_last_id;
    END IF;
    
    -- Remove spans fully covered by the new span
    DELETE FROM span
     WHERE attribute_id = p_attribute_id
       AND type = 'continuous'
       AND min_value >= p_new_min
       AND max_value <= p_new_max;
       
    INSERT INTO span(attribute_id, label, type, min_value, max_value)
    VALUES(p_attribute_id, p_label, 'continuous', p_new_min, p_new_max);
END;
$$;

## update_ventry_position

In [ ]:
%%sql

CREATE OR REPLACE FUNCTION update_ventry_position(
    p_variant_id INTEGER,
    p_position   INTEGER
)
RETURNS INTEGER
LANGUAGE plpgsql
AS $$
DECLARE
    v_count    INTEGER;
    v_position INTEGER;
BEGIN
    SELECT COUNT(*) INTO v_count
      FROM variant_entry
     WHERE variant_id = p_variant_id;
    
    IF p_position IS NULL THEN
        v_position := v_count;
    ELSE
        v_position := p_position;
        IF v_position < 0 THEN 
            v_position := 0;
        END IF;
        IF v_position > v_count THEN
            v_position := v_count;
        END IF;
        UPDATE variant_entry
           SET causation_index = causation_index + 1
         WHERE variant_id = p_variant_id
           AND causation_index >= v_position;
    END IF;
    RETURN v_position;
END;
$$;

## Link Attribute to Variant

In [ ]:
%%sql

CREATE OR REPLACE PROCEDURE link_attr_to_variant(
    p_variant_id   INTEGER,
    p_attribute_id INTEGER,
    p_name         VARCHAR,
    p_position     INTEGER DEFAULT NULL
)
LANGUAGE plpgsql
AS $$
DECLARE
    v_variant_entry_id INTEGER;
    v_position         INTEGER;
BEGIN
    -- Check if a variant entry with the same name already exists.
    IF EXISTS (
        SELECT 1 FROM variant_entry
         WHERE variant_id = p_variant_id
           AND name = p_name
    ) THEN
        RAISE EXCEPTION 'Variant entry with name "%" already exists for variant %', p_name, p_variant_id;
    END IF;

    v_position := update_ventry_position(p_variant_id, p_position);

    INSERT INTO variant_entry(name, variant_id, causation_index)
    VALUES (p_name, p_variant_id, v_position)
    RETURNING id INTO v_variant_entry_id;

    INSERT INTO attribute_ref(variant_entry_id, attribute_id)
    VALUES (v_variant_entry_id, p_attribute_id);
END;
$$;

## Link Variant to Variant

In [ ]:
%%sql

CREATE OR REPLACE PROCEDURE link_variant_to_variant(
    p_parent_variant_id INTEGER,
    p_name              VARCHAR,
    p_position          INTEGER DEFAULT NULL
)
LANGUAGE plpgsql
AS $$
DECLARE
    v_variant_entry_id INTEGER;
    v_position         INTEGER;
BEGIN
    -- Check if a variant entry with the same name already exists for the parent.
    IF EXISTS (
        SELECT 1 FROM variant_entry
         WHERE variant_id = p_parent_variant_id
           AND name = p_name
    ) THEN
        RAISE EXCEPTION 'Variant entry with name "%" already exists for variant %', p_name, p_parent_variant_id;
    END IF;

    v_position := update_ventry_position(p_parent_variant_id, p_position);

    INSERT INTO variant_entry(name, variant_id, causation_index)
    VALUES (p_name, p_parent_variant_id, v_position)
    RETURNING id INTO v_variant_entry_id;

    INSERT INTO variant_ref(variant_entry_id, variant_id)
    VALUES (v_variant_entry_id, p_parent_variant_id);
END;
$$;

## get_effects_from_resolved_path

In [ ]:
%%sql

CREATE OR REPLACE FUNCTION get_effects_from_resolved_path(
    p_path int[][]
)
RETURNS TABLE(
    idx int,
    id int,
    name varchar,
    span_id int,
    relationship text
)
AS $$
DECLARE
    n int;
    i int;
    j int;
    root_pa int;
    current_pa int;
BEGIN
    n := array_length(p_path, 1);
    IF n IS NULL OR n < 1 THEN
        RAISE EXCEPTION 'p_path must contain at least one element';
    END IF;

    FOR i IN REVERSE n..1 LOOP
        SELECT pa.id
          INTO root_pa
          FROM possible_attr pa
         WHERE pa.parent_possible_attr_id IS NULL
           AND pa.variant_entry_id = p_path[i][2]
           AND pa.span_id IS NOT DISTINCT FROM p_path[i][1]
         LIMIT 1;
        IF root_pa IS NULL THEN
            CONTINUE;
        END IF;
        current_pa := root_pa;
        FOR j IN i+1..n LOOP
            SELECT pa.id
              INTO current_pa
              FROM possible_attr pa
             WHERE pa.parent_possible_attr_id = current_pa
               AND pa.variant_entry_id = p_path[j][2]
               AND pa.span_id IS NOT DISTINCT FROM p_path[j][1]
             LIMIT 1;
            IF current_pa IS NULL THEN
                EXIT;
            END IF;
        END LOOP;
        IF current_pa IS NOT NULL THEN
            RETURN QUERY
            SELECT (n - i + 1) AS idx,
                   e.id,
                   e.name,
                   e.span_id,
                   'activates' AS relationship
              FROM effect e
             WHERE e.activating_possible_attr_id = current_pa
             ORDER BY e.id;
            RETURN QUERY
            SELECT (n - i + 1) AS idx,
                   e.id,
                   e.name,
                   e.span_id,
                   'effected_by' AS relationship
              FROM effect e
             WHERE e.to_modify_possible_attr_id = current_pa
             ORDER BY e.id;
        END IF;
    END LOOP;
    RETURN;
END;
$$ LANGUAGE plpgsql;

## Same Slot

In [ ]:
%%sql

CREATE OR REPLACE FUNCTION same_slot(
    p_candidate_id INTEGER,
    p_reference_id INTEGER
)
RETURNS BOOLEAN
LANGUAGE plpgsql
AS $$
DECLARE
    candidate_slot  INTEGER;
    reference_slot  INTEGER;
BEGIN
    -- If the candidate is a fallback (no possible_attr row), use abs(id)
    IF p_candidate_id < 0 THEN
       candidate_slot := -p_candidate_id;
    ELSE
       SELECT COALESCE(parent_possible_attr_id, id)
         INTO candidate_slot
         FROM possible_attr
        WHERE id = p_candidate_id;
    END IF;

    -- Same for the reference.
    IF p_reference_id < 0 THEN
       reference_slot := -p_reference_id;
    ELSE
       SELECT COALESCE(parent_possible_attr_id, id)
         INTO reference_slot
         FROM possible_attr
        WHERE id = p_reference_id;
    END IF;

    RETURN candidate_slot = reference_slot;
END;
$$;

## Variant

### Add

In [ ]:
%%sql

CREATE OR REPLACE PROCEDURE add_variant(
    in_variant_name VARCHAR,
    in_entries   JSON
)
LANGUAGE plpgsql
AS $$
DECLARE
    new_variant_id      INTEGER;
    i                   INTEGER := 0;
    key_count           INTEGER;
    attr_id             INTEGER;
    attr_name           VARCHAR;
    ventry_name         VARCHAR;
    element             JSON;
    element_type        TEXT;
    sub_variant_id      INTEGER;
    v_variant_entry_id  INTEGER;
BEGIN
    -- Use existing variant or insert it.
    SELECT id INTO new_variant_id
      FROM variant
     WHERE name = in_variant_name
     LIMIT 1;
    IF new_variant_id IS NULL THEN
        INSERT INTO variant(name)
        VALUES (in_variant_name)
        RETURNING id INTO new_variant_id;
    END IF;

    IF in_entries IS NOT NULL THEN
        key_count := json_array_length(in_entries);
        WHILE i < key_count LOOP
            element := in_entries -> i;
            element_type := json_typeof(element);
            IF element_type = 'array' THEN
                IF element ->> 0 = 'v' THEN
                    -- Subvariant: [ 'v', subvariant_name, ?optional_variant_entry_name ]
                    IF json_array_length(element) >= 3 THEN
                        ventry_name := element ->> 2;
                    ELSE
                        ventry_name := element ->> 1;
                    END IF;
                    SELECT id INTO sub_variant_id
                      FROM variant
                     WHERE name = element ->> 1
                     LIMIT 1;
                    IF sub_variant_id IS NULL THEN
                        INSERT INTO variant(name)
                        VALUES (element ->> 1)
                        RETURNING id INTO sub_variant_id;
                    END IF;
                    CALL link_variant_to_variant(new_variant_id, ventry_name, NULL);
                    SELECT id INTO v_variant_entry_id
                      FROM variant_entry
                     WHERE variant_id = new_variant_id
                       AND name = ventry_name;
                    UPDATE variant_ref
                      SET variant_id = sub_variant_id
                     WHERE variant_entry_id = v_variant_entry_id;
                ELSIF element ->> 0 = 'a' THEN
                    -- Attribute: [ 'a', attribute_name, ?optional_variant_entry_name ]
                    attr_name := element ->> 1;
                    IF json_array_length(element) >= 3 THEN
                        ventry_name := element ->> 2;
                    ELSE
                        ventry_name := attr_name;
                    END IF;
                    SELECT id INTO attr_id
                      FROM attribute
                     WHERE name = attr_name
                     LIMIT 1;
                    IF attr_id IS NULL THEN
                        RAISE EXCEPTION 'Attribute "%" not found', attr_name;
                    END IF;
                    CALL link_attr_to_variant(new_variant_id, attr_id, ventry_name, NULL);
                ELSE
                    RAISE EXCEPTION 'Invalid entry type: %', element ->> 0;
                END IF;
            ELSE
                -- Treat a simple string as an attribute.
                attr_name := element::text;
                ventry_name := attr_name;
                SELECT id INTO attr_id
                  FROM attribute
                 WHERE name = attr_name
                 LIMIT 1;
                IF attr_id IS NULL THEN
                    RAISE EXCEPTION 'Attribute "%" not found', attr_name;
                END IF;
                CALL link_attr_to_variant(new_variant_id, attr_id, ventry_name, NULL);
            END IF;
            i := i + 1;
        END LOOP;
    END IF;
END;
$$;

### Get

In [ ]:
%%sql

CREATE OR REPLACE FUNCTION get_variant(
  p_variant_name VARCHAR,
  p_max_depth INTEGER
)
RETURNS TABLE (
  depth       INTEGER,
  address     TEXT,
  activates   JSON,
  effected_by JSON
)
LANGUAGE plpgsql
AS $$
BEGIN
  RETURN QUERY
  WITH RECURSIVE
    root_variant AS (
      SELECT id 
      FROM variant 
      WHERE name = p_variant_name
    ),
    rec AS (
      -- Base step: initialize with nodes directly associated with the root.
      -- For a variant_ref the resolved_path element is [NULL, variant_entry_id].
      SELECT 
         ve.id AS variant_entry_id,
         NULL::int AS attribute_id,
         ve.name AS variant_entry_name,
         ve.causation_index,
         ARRAY[ve.causation_index] AS flattened_path,
         '[]'::jsonb AS addr_prefix,
         ve.name AS pending,
         ve.variant_id AS current_variant_id,
         -ve.id AS resolved_id,
         1 AS cur_depth,
         ARRAY[ARRAY[NULL::int, ve.id]] AS resolved_path,
         'ref' AS entry_type
      FROM variant_entry ve
      JOIN variant_ref vr ON vr.variant_entry_id = ve.id
      WHERE vr.variant_id = (SELECT id FROM root_variant)
      
      UNION ALL
      
      -- Also include base attribute nodes from the root.
      SELECT 
         ve.id AS variant_entry_id,
         ar.attribute_id,
         ve.name AS variant_entry_name,
         ve.causation_index,
         ARRAY[ve.causation_index] AS flattened_path,
         '[]'::jsonb AS addr_prefix,
         ve.name AS pending,
         ve.variant_id AS current_variant_id,
         -ve.id AS resolved_id,
         1 AS cur_depth,
         ARRAY[ARRAY[NULL::int, ve.id]] AS resolved_path,
         'attr' AS entry_type
      FROM variant_entry ve
      JOIN attribute_ref ar ON ar.variant_entry_id = ve.id
      WHERE ve.variant_id = (SELECT id FROM root_variant)
      
      UNION ALL
      
      -- Recursive step: from an attribute node, extend via a subvariant span to a child.
      SELECT 
         child_ve.id AS variant_entry_id,
         ar_child.attribute_id,
         child_ve.name AS variant_entry_name,
         child_ve.causation_index,
         r.flattened_path || child_ve.causation_index AS flattened_path,
         CASE 
           WHEN r.addr_prefix = '[]'::jsonb THEN jsonb_build_array(jsonb_build_array(r.pending, cand.span_label))
           ELSE r.addr_prefix || jsonb_build_array(jsonb_build_array(r.pending, cand.span_label))
         END AS addr_prefix,
         child_ve.name AS pending,
         child_ve.variant_id AS current_variant_id,
         -child_ve.id AS resolved_id,
         r.cur_depth + 1 AS cur_depth,
         r.resolved_path || ARRAY[ARRAY[cand.span_id, child_ve.id]] AS resolved_path,
         'attr' AS entry_type
      FROM rec r
      JOIN LATERAL (
         SELECT s.id AS span_id,
                s.label AS span_label,
                vr.variant_entry_id AS candidate_variant_entry_id,
                vr.variant_id AS child_variant_id
         FROM span s
         JOIN subvariant_span ss ON ss.span_id = s.id
         JOIN variant_ref vr ON vr.variant_id = r.current_variant_id
         WHERE s.attribute_id = (
                   SELECT ar.attribute_id 
                   FROM attribute_ref ar 
                   WHERE ar.variant_entry_id = r.variant_entry_id
               )
         ORDER BY ss.id
         LIMIT 1
      ) cand ON true
      JOIN variant_entry child_ve ON child_ve.id = cand.candidate_variant_entry_id
      JOIN attribute_ref ar_child ON ar_child.variant_entry_id = child_ve.id
      WHERE r.entry_type = 'attr' AND r.cur_depth < p_max_depth
    ),
    final AS (
      SELECT 
         rec.flattened_path,
         rec.cur_depth,
         rec.addr_prefix,
         rec.pending,
         rec.resolved_path
      FROM rec
    )
  SELECT 
    final.cur_depth AS depth,
    CASE 
      WHEN final.addr_prefix = '[]'::jsonb THEN jsonb_build_array(final.pending)
      ELSE final.addr_prefix || jsonb_build_array(final.pending)
    END::text AS address,
    eff.activates,
    eff.effected_by
  FROM final
  LEFT JOIN LATERAL (
    SELECT 
      json_agg(eff1.name ORDER BY eff1.name) FILTER (WHERE eff1.relationship = 'activates') AS activates,
      json_agg(eff1.name ORDER BY eff1.name) FILTER (WHERE eff1.relationship = 'effected_by') AS effected_by
    FROM get_effects_from_resolved_path(final.resolved_path) eff1
  ) eff ON true
  ORDER BY final.flattened_path;
END;
$$;

### Delete

In [ ]:
%%sql

CREATE OR REPLACE PROCEDURE delete_variant(p_variant_name varchar)
LANGUAGE plpgsql
AS
$$
BEGIN
  DELETE FROM variant WHERE name = p_variant_name;
END;
$$;

## Add Entity

In [ ]:
%%sql

CREATE OR REPLACE PROCEDURE add_entity(
    in_name VARCHAR(255),
    in_variant_id INT
)
LANGUAGE plpgsql
AS $$
BEGIN
    INSERT INTO entity(name, variant_id)
    VALUES (in_name, in_variant_id);
END;
$$;

## Roll Discrete Variant Attribute Value

In [ ]:
%%sql

CREATE OR REPLACE FUNCTION roll_discrete_varattr(
    p_variant_entry_ids int[][],
    p_entity_state_id INTEGER,
    p_exclude_span_id INTEGER
)
RETURNS INTEGER
LANGUAGE plpgsql
AS $$
DECLARE
    v_span_id INTEGER;
    v_max_double CONSTANT double precision := 1e308;
    leaf_ventry_id INTEGER;
BEGIN
    -- Extract the leaf variant_entry from the path tuple ([span_id, variant_entry_id])
    leaf_ventry_id := p_variant_entry_ids[array_length(p_variant_entry_ids, 1)][2];

    WITH
    BaseSpans AS (
        SELECT s.id AS span_id,
               ve.id AS variant_entry_id,
               NULL::INTEGER AS subvariant_span_id,
               s.weight AS base_weight
        FROM variant_entry ve
        JOIN attribute_ref ar ON ar.variant_entry_id = ve.id
        JOIN span s ON s.attribute_id = ar.attribute_id
        WHERE ve.id = leaf_ventry_id
          AND s.type = 'discrete'
          AND (p_exclude_span_id = 0 OR s.id <> p_exclude_span_id)
          AND s.id NOT IN (SELECT span_id FROM add_span_eft)
    ),
    Effects AS (
         SELECT *
         FROM get_effects_from_resolved_path(p_variant_entry_ids)
         WHERE relationship = 'effected_by'
    ),
    ActiveEffects AS (
         SELECT e.id AS effect_id, e.span_id
         FROM effect e
         JOIN Effects g ON e.id = g.id
         JOIN attr_value av ON av.possible_attr_id = e.activating_possible_attr_id
         WHERE av.entity_state_id = p_entity_state_id
           AND av.span_id = e.span_id
    ),
    InactiveSpans AS (
         SELECT DISTINCT rse.span_id
         FROM remove_span_eft rse
         JOIN ActiveEffects ae ON ae.effect_id = rse.effect_id
    ),
    ActivatedSpans AS (
         SELECT DISTINCT
                ase.span_id,
                NULL::INTEGER AS variant_entry_id,
                NULL::INTEGER AS subvariant_span_id,
                0.0 AS base_weight
         FROM add_span_eft ase
         JOIN ActiveEffects ae ON ae.effect_id = ase.effect_id
    ),
    DeltaWeights AS (
         SELECT dwe.span_id, SUM(dwe.delta_weight) AS total_delta
         FROM delta_weight_eft dwe
         JOIN ActiveEffects ae ON ae.effect_id = dwe.effect_id
         GROUP BY dwe.span_id
    ),
    AllRelevantSpans AS (
         SELECT b.span_id, b.variant_entry_id, b.subvariant_span_id, b.base_weight
         FROM BaseSpans b
         WHERE b.span_id NOT IN (SELECT span_id FROM InactiveSpans)
         UNION
         SELECT a.span_id, a.variant_entry_id, a.subvariant_span_id, a.base_weight
         FROM ActivatedSpans a
         WHERE a.span_id NOT IN (SELECT span_id FROM InactiveSpans)
    ),
    FinalSpans AS (
         SELECT ars.span_id,
                ars.variant_entry_id,
                ars.subvariant_span_id,
                COALESCE(ars.base_weight, 0) + COALESCE(dw.total_delta, 0) AS effective_weight
         FROM AllRelevantSpans ars
         LEFT JOIN DeltaWeights dw ON dw.span_id = ars.span_id
    ),
    SpanCounts AS (
         SELECT ns.num_spans,
                SUM(fs.effective_weight / ns.num_spans) AS total_weight
         FROM FinalSpans fs
         CROSS JOIN (
             SELECT COUNT(*)::double precision AS num_spans
             FROM FinalSpans
             WHERE effective_weight > 0
         ) ns
         WHERE fs.effective_weight > 0
         GROUP BY ns.num_spans
    ),
    AdjustedSpans AS (
         SELECT fs.span_id,
                fs.variant_entry_id,
                fs.subvariant_span_id,
                fs.effective_weight,
                sc.num_spans,
                sc.total_weight,
                CASE 
                  WHEN fs.effective_weight < 1 THEN ceil((fs.effective_weight * v_max_double) / (sc.total_weight * sc.num_spans))
                  ELSE ceil((fs.effective_weight / (sc.total_weight * sc.num_spans)) * v_max_double)
                END AS adjusted_weight
         FROM FinalSpans fs
         CROSS JOIN SpanCounts sc
         WHERE fs.effective_weight > 0
    ),
    Running AS (
         SELECT span_id,
                variant_entry_id,
                subvariant_span_id,
                adjusted_weight,
                SUM(adjusted_weight) OVER (ORDER BY span_id) AS running_total
         FROM AdjustedSpans
    ),
    OrderedSpansCTE AS (
         SELECT span_id,
                variant_entry_id,
                subvariant_span_id,
                adjusted_weight AS contextual_weight,
                running_total,
                LAG(running_total, 1, 0) OVER (ORDER BY span_id) AS prev_running_total
         FROM Running
    ),
    TotalAdjusted AS (
         SELECT MAX(running_total) AS v_total_weight
         FROM OrderedSpansCTE
    ),
    Rng AS (
         SELECT t.v_total_weight,
                floor(random() * t.v_total_weight) AS v_random_pick
         FROM TotalAdjusted t
    )
    SELECT o.span_id
      INTO v_span_id
      FROM OrderedSpansCTE o, Rng
      WHERE Rng.v_random_pick >= o.prev_running_total
        AND Rng.v_random_pick < o.running_total
      LIMIT 1;

    RETURN v_span_id;
END;
$$;

## Gamma Range

In [ ]:
%%sql

CREATE OR REPLACE FUNCTION gamma_rng(shape double precision)
RETURNS double precision
LANGUAGE plpgsql
AS $$
DECLARE
    d double precision;
    c double precision;
    x double precision;
    v double precision;
    u double precision;
BEGIN
    IF shape < 1 THEN
        -- Use the transformation: Gamma(shape) = Gamma(shape+1)*U^(1/shape)
        RETURN gamma_rng(shape + 1) * power(random(), 1.0 / shape);
    END IF;
    d := shape - 1.0/3.0;
    c := 1.0 / sqrt(9.0 * d);
    LOOP
        -- Generate a standard normal via Box–Muller:
        x := sqrt(-2 * ln(random())) * cos(2 * 3.141592653589793 * random());
        v := 1 + c * x;
        IF v <= 0 THEN
            CONTINUE;
        END IF;
        v := v * v * v;
        u := random();
        IF u < 1 - 0.0331 * x * x * x * x THEN
            RETURN d * v;
        END IF;
        IF ln(u) < 0.5 * x * x + d * (1 - v + ln(v)) THEN
            RETURN d * v;
        END IF;
    END LOOP;
END;
$$;

## Generate Weighted Random Number

In [ ]:
%%sql

-- using a fast Marsaglia–Tsang method
CREATE OR REPLACE FUNCTION weighted_random(
    min_val double precision,                   -- Minimum of desired range.
    max_val double precision,                   -- Maximum of desired range (must be > min_val).
    mode_val double precision,                  -- Desired mode (peak) in the same units.
    skew double precision DEFAULT 0,            -- Skew adjustment; between -1 and 1 only, 0 = no skew.
    concentration double precision DEFAULT 4    -- Concentration (k); should be >2 for a proper unimodal shape.
) RETURNS double precision
LANGUAGE plpgsql
AS $$
DECLARE
    m_norm      double precision;  -- Normalized desired mode in [0,1].
    alpha       double precision;
    beta        double precision;
    x_val       double precision;
    y_val       double precision;
    beta_sample double precision;
BEGIN
    IF min_val >= max_val THEN
        RAISE EXCEPTION 'min_val must be less than max_val';
    END IF;
    IF concentration < 0 THEN
        RAISE EXCEPTION 'concentration must be non-negative';
    END IF;
    IF concentration <= 2 THEN
        -- When concentration <= 2 the Beta isn't unimodal; fallback to uniform.
        RETURN min_val + random() * (max_val - min_val);
    END IF;

    -- Normalize the desired mode to [0,1].
    m_norm := (mode_val - min_val) / (max_val - min_val);

    -- Parameterize α and β so that (α-1)/(α+β-2) equals m_norm.
    alpha := m_norm * (concentration - 2) + 1;
    beta  := (1 - m_norm) * (concentration - 2) + 1;

    -- Apply skew correction if desired.
    IF skew <> 0 THEN
        alpha := alpha * (1 + skew);
        beta  := beta  * (1 - skew);
    END IF;

    -- Generate a Beta(α,β) random variable via the Gamma method.
    x_val := gamma_rng(alpha);
    y_val := gamma_rng(beta);
    beta_sample := x_val / (x_val + y_val);

    -- Scale the [0,1] Beta sample to [min_val, max_val].
    RETURN min_val + beta_sample * (max_val - min_val);
END;
$$;

## Roll Continuous Variant Attribute Value

In [ ]:
%%sql

CREATE OR REPLACE FUNCTION roll_continuous_varattr(
    p_variant_entry_ids int[][],
    p_entity_state_id   INTEGER,
    p_exclude_span_id   INTEGER
)
RETURNS TABLE(chosen_span_id integer, chosen_value double precision)
LANGUAGE plpgsql
AS $$
DECLARE
    v_attribute_id         INTEGER;
    v_decimals             INTEGER;
    v_min                  DOUBLE PRECISION;
    v_max                  DOUBLE PRECISION;
    v_mode                 DOUBLE PRECISION;
    v_concentration        DOUBLE PRECISION;
    v_skew                 DOUBLE PRECISION;
    v_total_delta_mode     DOUBLE PRECISION := 0;
    v_total_delta_conc     DOUBLE PRECISION := 0;
    v_total_delta_skew     DOUBLE PRECISION := 0;
    v_eff_min              DOUBLE PRECISION;
    v_eff_max              DOUBLE PRECISION;
    v_eff_mode             DOUBLE PRECISION;
    v_result               DOUBLE PRECISION;
    v_clamped_result       DOUBLE PRECISION;
    v_chosen_span_id       INTEGER;
    dummy_eff_min          DOUBLE PRECISION;
    dummy_eff_max          DOUBLE PRECISION;
    v_variant_entry_id     INTEGER;
    leaf_ventry_id         INTEGER;
BEGIN
    -- Extract the leaf variant_entry (second element of the last tuple)
    leaf_ventry_id := p_variant_entry_ids[array_length(p_variant_entry_ids,1)][2];

    SELECT ar.attribute_id, a.decimals, a.min_value, a.max_value, a.mode_value, 
           a.concentration, a.skew, ar.variant_entry_id
      INTO v_attribute_id, v_decimals, v_min, v_max, v_mode, v_concentration, v_skew, v_variant_entry_id
      FROM attribute_ref ar
      JOIN attribute a ON a.id = ar.attribute_id
     WHERE ar.variant_entry_id = leaf_ventry_id
     LIMIT 1;

    IF v_attribute_id IS NULL THEN
        RAISE EXCEPTION 'No matching attribute found for variant_entry_id %', leaf_ventry_id;
    END IF;

    WITH Effects AS (
         SELECT *
         FROM get_effects_from_resolved_path(p_variant_entry_ids)
         WHERE relationship = 'effected_by'
    ),
    ActiveEffects AS (
         SELECT e.id AS effect_id, e.span_id, e.to_modify_possible_attr_id, e.variant_id
         FROM effect e
         JOIN Effects g ON e.id = g.id
         JOIN attr_value av ON av.possible_attr_id = e.activating_possible_attr_id
         WHERE av.entity_state_id = p_entity_state_id
           AND av.span_id = e.span_id
    )
    SELECT COALESCE(SUM(dpe.delta_mode), 0),
           COALESCE(SUM(dpe.delta_conc), 0),
           COALESCE(SUM(dpe.delta_skew), 0)
      INTO v_total_delta_mode, v_total_delta_conc, v_total_delta_skew
      FROM delta_param_eft dpe
      JOIN ActiveEffects ae ON ae.effect_id = dpe.effect_id
      JOIN possible_attr pa ON pa.id = ae.to_modify_possible_attr_id
      WHERE pa.variant_entry_id = v_variant_entry_id
        AND ae.variant_id = (SELECT variant_id FROM variant_entry WHERE id = leaf_ventry_id);

    IF (v_total_delta_conc <> 0 OR v_total_delta_skew <> 0) THEN
        v_eff_min  := (v_min + v_total_delta_mode) * (1 + v_total_delta_conc + v_total_delta_skew);
        v_eff_max  := (v_max + v_total_delta_mode) * (1 + v_total_delta_conc + v_total_delta_skew);
        v_eff_mode := (v_mode + v_total_delta_mode) * (1 + v_total_delta_conc + v_total_delta_skew);
    ELSE
        v_eff_min  := v_min + v_total_delta_mode;
        v_eff_max  := v_max + v_total_delta_mode;
        v_eff_mode := v_mode + v_total_delta_mode;
    END IF;

    IF v_eff_max < v_eff_min THEN
        v_result  := v_eff_min;
        v_eff_min := v_eff_max;
        v_eff_max := v_result;
    END IF;

    v_result := weighted_random(
                   v_eff_min, 
                   v_eff_max, 
                   v_eff_mode, 
                   COALESCE(v_skew, 0),
                   COALESCE(v_concentration, 0)
               );
    v_result := round(v_result::numeric, v_decimals)::double precision;

    IF v_result < v_eff_min THEN
        v_clamped_result := v_eff_min;
    ELSIF v_result > v_eff_max THEN
        v_clamped_result := v_eff_max;
    ELSE
        v_clamped_result := v_result;
    END IF;

    IF p_exclude_span_id IS NOT NULL AND p_exclude_span_id <> 0 THEN
        SELECT s.id,
               CASE 
                 WHEN (v_total_delta_conc <> 0 OR v_total_delta_skew <> 0)
                 THEN (s.min_value + v_total_delta_mode) * (1 + v_total_delta_conc + v_total_delta_skew)
                 ELSE (s.min_value + v_total_delta_mode)
               END,
               CASE 
                 WHEN (v_total_delta_conc <> 0 OR v_total_delta_skew <> 0)
                 THEN (s.max_value + v_total_delta_mode) * (1 + v_total_delta_conc + v_total_delta_skew)
                 ELSE (s.max_value + v_total_delta_mode)
               END
          INTO v_chosen_span_id, dummy_eff_min, dummy_eff_max
          FROM span s
          JOIN possible_attr pa ON pa.span_id = s.id
         WHERE s.attribute_id = v_attribute_id
           AND s.type = 'continuous'
           AND s.id <> p_exclude_span_id
           AND pa.variant_entry_id = v_variant_entry_id
           AND (CASE 
                  WHEN (v_total_delta_conc <> 0 OR v_total_delta_skew <> 0)
                  THEN (s.min_value + v_total_delta_mode) * (1 + v_total_delta_conc + v_total_delta_skew)
                  ELSE (s.min_value + v_total_delta_mode)
                END) <= v_clamped_result
           AND (CASE 
                  WHEN (v_total_delta_conc <> 0 OR v_total_delta_skew <> 0)
                  THEN (s.max_value + v_total_delta_mode) * (1 + v_total_delta_conc + v_total_delta_skew)
                  ELSE (s.max_value + v_total_delta_mode)
                END) > v_clamped_result
          LIMIT 1;
    ELSE
        SELECT s.id,
               CASE 
                 WHEN (v_total_delta_conc <> 0 OR v_total_delta_skew <> 0)
                 THEN (s.min_value + v_total_delta_mode) * (1 + v_total_delta_conc + v_total_delta_skew)
                 ELSE (s.min_value + v_total_delta_mode)
               END,
               CASE 
                 WHEN (v_total_delta_conc <> 0 OR v_total_delta_skew <> 0)
                 THEN (s.max_value + v_total_delta_mode) * (1 + v_total_delta_conc + v_total_delta_skew)
                 ELSE (s.max_value + v_total_delta_mode)
               END
          INTO v_chosen_span_id, dummy_eff_min, dummy_eff_max
          FROM span s
          JOIN possible_attr pa ON pa.span_id = s.id
         WHERE s.attribute_id = v_attribute_id
           AND s.type = 'continuous'
           AND pa.variant_entry_id = v_variant_entry_id
           AND (CASE 
                  WHEN (v_total_delta_conc <> 0 OR v_total_delta_skew <> 0)
                  THEN (s.min_value + v_total_delta_mode) * (1 + v_total_delta_conc + v_total_delta_skew)
                  ELSE (s.min_value + v_total_delta_mode)
                END) <= v_clamped_result
           AND (CASE 
                  WHEN (v_total_delta_conc <> 0 OR v_total_delta_skew <> 0)
                  THEN (s.max_value + v_total_delta_mode) * (1 + v_total_delta_conc + v_total_delta_skew)
                  ELSE (s.max_value + v_total_delta_mode)
                END) > v_clamped_result
          LIMIT 1;
    END IF;

    chosen_span_id := v_chosen_span_id;
    chosen_value   := v_clamped_result;
    RETURN NEXT;
END;
$$;

## Generate Entity State

In [ ]:
%%sql

CREATE OR REPLACE PROCEDURE generate_entity_state(
    IN p_entity_name VARCHAR,
    IN p_time        DOUBLE PRECISION
)
LANGUAGE plpgsql
AS $$
DECLARE
    v_entity_id               INTEGER;
    v_root_variant_id         INTEGER;
    v_entity_state_id         INTEGER;
    v_is_regenerate           BOOLEAN;
    v_current_variant         INTEGER;
    v_trigger_ss              INTEGER;
    v_parent_possible_attr_id INTEGER;
    v_parent_path             int[][];  -- Each element: [span_id, variant_entry_id]
    v_queue_id                INTEGER;
    cur_possible_attr_id      INTEGER;
    v_current_ventry_id       INTEGER;
    cur_attr_type             TEXT;
    v_sub_variant_id          INTEGER;
    v_existing_attr_val_id    INTEGER;
    v_existing_span_id        INTEGER;
    v_lock_count              INTEGER := 0;
    v_new_span_id             INTEGER;
    v_new_numeric             DOUBLE PRECISION;
    v_used_span_id            INTEGER;
    rec                       RECORD;
    new_path                  int[][];  -- New path after appending current element
BEGIN
    -- Lookup entity.
    SELECT id, variant_id 
      INTO v_entity_id, v_root_variant_id
      FROM entity
     WHERE name = p_entity_name;
    IF v_entity_id IS NULL THEN
       RAISE EXCEPTION 'No entity found with name=%', p_entity_name;
    END IF;

    -- Check for existing state.
    SELECT id 
      INTO v_entity_state_id
      FROM entity_state
     WHERE entity_id = v_entity_id
       AND "time" = p_time
     LIMIT 1;
    IF v_entity_state_id IS NOT NULL THEN
        v_is_regenerate := TRUE;
    ELSE
        INSERT INTO entity_state(entity_id, "time")
        VALUES (v_entity_id, p_time)
        RETURNING id INTO v_entity_state_id;
        v_is_regenerate := FALSE;
    END IF;

    -- Create temporary queue carrying a 2D integer array path.
    DROP TABLE IF EXISTS _variant_queue;
    CREATE TEMPORARY TABLE _variant_queue (
        queue_id serial PRIMARY KEY,
        variant_id INTEGER,
        subvariant_span_id INTEGER,
        parent_possible_attr_id INTEGER,
        parent_path int[][] 
    ) ON COMMIT DROP;
    INSERT INTO _variant_queue (variant_id, subvariant_span_id, parent_possible_attr_id, parent_path)
      VALUES (v_root_variant_id, NULL, NULL, '{}'::int[][]);

    -- Temporary table for variant entries.
    DROP TABLE IF EXISTS _variant_attrs;
    CREATE TEMPORARY TABLE _variant_attrs (
        ve_id INTEGER,
        attr_type TEXT,
        is_variant_ref BOOLEAN,
        subvariant_variant_id INTEGER
    ) ON COMMIT DROP;

    -- Process the queue (LIFO).
    LOOP
        SELECT queue_id, variant_id, subvariant_span_id, parent_possible_attr_id, parent_path
          INTO v_queue_id, v_current_variant, v_trigger_ss, v_parent_possible_attr_id, v_parent_path
          FROM _variant_queue
         ORDER BY queue_id DESC
         LIMIT 1;
        EXIT WHEN NOT FOUND;
        DELETE FROM _variant_queue WHERE queue_id = v_queue_id;

        TRUNCATE _variant_attrs;
        INSERT INTO _variant_attrs (ve_id, attr_type, is_variant_ref, subvariant_variant_id)
          SELECT ve.id,
                 a.type,
                 (CASE WHEN vr.variant_id IS NOT NULL THEN TRUE ELSE FALSE END),
                 vr.variant_id
            FROM variant_entry ve
            LEFT JOIN attribute_ref ar ON ve.id = ar.variant_entry_id
            LEFT JOIN attribute a ON ar.attribute_id = a.id
            LEFT JOIN variant_ref vr ON ve.id = vr.variant_entry_id
           WHERE ve.variant_id = v_current_variant
           ORDER BY ve.causation_index;

        FOR rec IN SELECT * FROM _variant_attrs LOOP
            v_current_ventry_id := rec.ve_id;
            IF rec.is_variant_ref THEN
                -- For variant_ref, possible_attr.span_id must be NULL.
                SELECT id INTO cur_possible_attr_id
                  FROM possible_attr
                 WHERE variant_entry_id = v_current_ventry_id
                   AND ((parent_possible_attr_id IS NULL AND v_parent_possible_attr_id IS NULL)
                        OR parent_possible_attr_id = v_parent_possible_attr_id)
                   AND span_id IS NULL
                 LIMIT 1;
                IF cur_possible_attr_id IS NULL THEN
                    INSERT INTO possible_attr (variant_entry_id, parent_possible_attr_id, span_id)
                    VALUES (v_current_ventry_id, v_parent_possible_attr_id, NULL)
                    RETURNING id INTO cur_possible_attr_id;
                END IF;
                new_path := v_parent_path || ARRAY[ARRAY[NULL::integer, v_current_ventry_id]];
                INSERT INTO _variant_queue (variant_id, subvariant_span_id, parent_possible_attr_id, parent_path)
                VALUES (rec.subvariant_variant_id, NULL, cur_possible_attr_id, new_path)
                ON CONFLICT DO NOTHING;
            ELSE
                -- For attribute_ref, we roll to get a valid span.
                IF v_is_regenerate THEN
                    SELECT id, span_id
                      INTO v_existing_attr_val_id, v_existing_span_id
                      FROM attr_value
                     WHERE entity_state_id = v_entity_state_id
                       AND possible_attr_id = (
                           SELECT id
                             FROM possible_attr
                            WHERE variant_entry_id = v_current_ventry_id
                              AND ((parent_possible_attr_id IS NULL AND v_parent_possible_attr_id IS NULL)
                                   OR parent_possible_attr_id = v_parent_possible_attr_id)
                              AND span_id IS NOT NULL
                            LIMIT 1
                       )
                     LIMIT 1;
                    IF v_existing_attr_val_id IS NOT NULL THEN
                        SELECT count(*) INTO v_lock_count
                          FROM attr_val_lock
                         WHERE locked_attr_val_id = v_existing_attr_val_id;
                        IF v_lock_count > 0 THEN
                            v_used_span_id := v_existing_span_id;
                        ELSE
                            IF rec.attr_type = 'discrete' THEN
                                v_new_span_id := roll_discrete_varattr(
                                    v_parent_path || ARRAY[ARRAY[NULL::integer, v_current_ventry_id]],
                                    v_entity_state_id, 0);
                                UPDATE attr_value
                                   SET span_id = v_new_span_id
                                 WHERE id = v_existing_attr_val_id;
                                v_used_span_id := v_new_span_id;
                            ELSE
                                SELECT t.chosen_span_id, t.chosen_value
                                  INTO v_new_span_id, v_new_numeric
                                  FROM roll_continuous_varattr(
                                    v_parent_path || ARRAY[ARRAY[NULL::integer, v_current_ventry_id]],
                                    v_entity_state_id, 0) t
                                 LIMIT 1;
                                UPDATE attr_value
                                   SET span_id = v_new_span_id,
                                       numeric_value = v_new_numeric
                                 WHERE id = v_existing_attr_val_id;
                                v_used_span_id := v_new_span_id;
                            END IF;
                        END IF;
                    ELSE
                        IF rec.attr_type = 'discrete' THEN
                            v_new_span_id := roll_discrete_varattr(
                                    v_parent_path || ARRAY[ARRAY[NULL::integer, v_current_ventry_id]],
                                    v_entity_state_id, 0);
                            SELECT id INTO cur_possible_attr_id
                              FROM possible_attr
                             WHERE variant_entry_id = v_current_ventry_id
                               AND ((parent_possible_attr_id IS NULL AND v_parent_possible_attr_id IS NULL)
                                    OR parent_possible_attr_id = v_parent_possible_attr_id)
                               AND span_id = v_new_span_id
                             LIMIT 1;
                            IF cur_possible_attr_id IS NULL THEN
                                INSERT INTO possible_attr (variant_entry_id, parent_possible_attr_id, span_id)
                                VALUES (v_current_ventry_id, v_parent_possible_attr_id, v_new_span_id)
                                RETURNING id INTO cur_possible_attr_id;
                            END IF;
                            INSERT INTO attr_value(entity_state_id, numeric_value, span_id, possible_attr_id)
                            VALUES (v_entity_state_id, NULL, v_new_span_id, cur_possible_attr_id);
                            v_used_span_id := v_new_span_id;
                        ELSE
                            SELECT t.chosen_span_id, t.chosen_value
                              INTO v_new_span_id, v_new_numeric
                              FROM roll_continuous_varattr(
                                    v_parent_path || ARRAY[ARRAY[NULL::integer, v_current_ventry_id]],
                                    v_entity_state_id, 0) t
                             LIMIT 1;
                            SELECT id INTO cur_possible_attr_id
                              FROM possible_attr
                             WHERE variant_entry_id = v_current_ventry_id
                               AND ((parent_possible_attr_id IS NULL AND v_parent_possible_attr_id IS NULL)
                                    OR parent_possible_attr_id = v_parent_possible_attr_id)
                               AND span_id = v_new_span_id
                             LIMIT 1;
                            IF cur_possible_attr_id IS NULL THEN
                                INSERT INTO possible_attr (variant_entry_id, parent_possible_attr_id, span_id)
                                VALUES (v_current_ventry_id, v_parent_possible_attr_id, v_new_span_id)
                                RETURNING id INTO cur_possible_attr_id;
                            END IF;
                            INSERT INTO attr_value(entity_state_id, numeric_value, span_id, possible_attr_id)
                            VALUES (v_entity_state_id, v_new_numeric, v_new_span_id, cur_possible_attr_id);
                            v_used_span_id := v_new_span_id;
                        END IF;
                    END IF;
                ELSE
                    IF rec.attr_type = 'discrete' THEN
                        v_new_span_id := roll_discrete_varattr(
                                    v_parent_path || ARRAY[ARRAY[NULL::integer, v_current_ventry_id]],
                                    v_entity_state_id, 0);
                        SELECT id INTO cur_possible_attr_id
                          FROM possible_attr
                         WHERE variant_entry_id = v_current_ventry_id
                           AND ((parent_possible_attr_id IS NULL AND v_parent_possible_attr_id IS NULL)
                                OR parent_possible_attr_id = v_parent_possible_attr_id)
                           AND span_id = v_new_span_id
                         LIMIT 1;
                        IF cur_possible_attr_id IS NULL THEN
                            INSERT INTO possible_attr (variant_entry_id, parent_possible_attr_id, span_id)
                            VALUES (v_current_ventry_id, v_parent_possible_attr_id, v_new_span_id)
                            RETURNING id INTO cur_possible_attr_id;
                        END IF;
                        INSERT INTO attr_value(entity_state_id, numeric_value, span_id, possible_attr_id)
                        VALUES (v_entity_state_id, NULL, v_new_span_id, cur_possible_attr_id);
                        v_used_span_id := v_new_span_id;
                    ELSE
                        SELECT t.chosen_span_id, t.chosen_value
                          INTO v_new_span_id, v_new_numeric
                          FROM roll_continuous_varattr(
                                    v_parent_path || ARRAY[ARRAY[NULL::integer, v_current_ventry_id]],
                                    v_entity_state_id, 0) t
                         LIMIT 1;
                        SELECT id INTO cur_possible_attr_id
                          FROM possible_attr
                         WHERE variant_entry_id = v_current_ventry_id
                           AND ((parent_possible_attr_id IS NULL AND v_parent_possible_attr_id IS NULL)
                                OR parent_possible_attr_id = v_parent_possible_attr_id)
                           AND span_id = v_new_span_id
                         LIMIT 1;
                        IF cur_possible_attr_id IS NULL THEN
                            INSERT INTO possible_attr (variant_entry_id, parent_possible_attr_id, span_id)
                            VALUES (v_current_ventry_id, v_parent_possible_attr_id, v_new_span_id)
                            RETURNING id INTO cur_possible_attr_id;
                        END IF;
                        INSERT INTO attr_value(entity_state_id, numeric_value, span_id, possible_attr_id)
                        VALUES (v_entity_state_id, v_new_numeric, v_new_span_id, cur_possible_attr_id);
                        v_used_span_id := v_new_span_id;
                    END IF;
                END IF;
                SELECT id INTO cur_possible_attr_id
                  FROM possible_attr
                 WHERE variant_entry_id = v_current_ventry_id
                   AND ((parent_possible_attr_id IS NULL AND v_parent_possible_attr_id IS NULL)
                        OR parent_possible_attr_id = v_parent_possible_attr_id)
                   AND span_id = v_used_span_id
                 LIMIT 1;
                IF cur_possible_attr_id IS NULL THEN
                    INSERT INTO possible_attr (variant_entry_id, parent_possible_attr_id, span_id)
                    VALUES (v_current_ventry_id, v_parent_possible_attr_id, v_used_span_id)
                    RETURNING id INTO cur_possible_attr_id;
                END IF;
                new_path := v_parent_path || ARRAY[ARRAY[v_used_span_id, v_current_ventry_id]];
                SELECT variant_id
                  INTO v_sub_variant_id
                  FROM subvariant_span
                 WHERE span_id = v_used_span_id
                 LIMIT 1;
                IF v_sub_variant_id IS NOT NULL THEN
                    INSERT INTO _variant_queue (variant_id, subvariant_span_id, parent_possible_attr_id, parent_path)
                    VALUES (v_sub_variant_id, NULL, cur_possible_attr_id, new_path)
                    ON CONFLICT DO NOTHING;
                END IF;
            END IF;
        END LOOP;
    END LOOP;

    DROP TABLE IF EXISTS _variant_attrs;
    DROP TABLE IF EXISTS _variant_queue;
END;
$$;

## Get Entity State

In [ ]:
%%sql

CREATE OR REPLACE FUNCTION get_entity_state(
    p_entity_name VARCHAR,
    p_time        DOUBLE PRECISION
)
RETURNS TABLE (
    "index"       BIGINT,
    "address"     JSON,
    activates     JSON,
    effected_by   JSON,
    numeric_value DOUBLE PRECISION,
    span          VARCHAR,
    locked        BOOLEAN
)
LANGUAGE plpgsql
AS $$
BEGIN
  RETURN QUERY
  WITH RECURSIVE entity_info AS (
      -- Get entity_state and its variant.
      SELECT es.id AS entity_state_id,
             e.variant_id
      FROM entity_state es
      JOIN entity e ON e.id = es.entity_id
      WHERE e.name = p_entity_name
        AND es."time" = p_time
  ),
  base AS (
      -- Top-level possible_attr with d0 always having NULL span in the path.
      SELECT 
         pa.id AS possible_attr_id,
         ve.name AS attribute_name,
         ve.causation_index,
         ARRAY[[NULL::int, ve.id]]::int[][] AS variant_entry_path,
         av.id AS attr_val_id,
         av.numeric_value,
         pa.span_id,
         ei.entity_state_id
      FROM possible_attr pa
      JOIN variant_entry ve ON ve.id = pa.variant_entry_id
      JOIN entity_info ei ON ve.variant_id = ei.variant_id
      JOIN attr_value av ON av.possible_attr_id = pa.id
           AND av.entity_state_id = ei.entity_state_id
      WHERE pa.parent_possible_attr_id IS NULL
  ),
  recursive_flattened AS (
      SELECT * FROM base
      UNION ALL
      SELECT 
         child.possible_attr_id,
         child.attribute_name,
         child.causation_index,
         rf.variant_entry_path || ARRAY[[child.span_id, child.variant_entry_id]]::int[][] AS variant_entry_path,
         child.attr_val_id,
         child.numeric_value,
         child.span_id,
         child.entity_state_id
      FROM (
          SELECT 
             pa.id AS possible_attr_id,
             ve.name AS attribute_name,
             ve.causation_index,
             pa.parent_possible_attr_id,
             av.id AS attr_val_id,
             av.numeric_value,
             pa.span_id,
             ve.id AS variant_entry_id,
             ei.entity_state_id
          FROM possible_attr pa
          JOIN variant_entry ve ON ve.id = pa.variant_entry_id
          JOIN attr_value av ON av.possible_attr_id = pa.id
               AND av.entity_state_id = (SELECT entity_state_id FROM entity_info)
          JOIN entity_info ei ON TRUE
          WHERE pa.parent_possible_attr_id IS NOT NULL
      ) child
      JOIN recursive_flattened rf ON child.parent_possible_attr_id = rf.possible_attr_id
  ),
  ordered AS (
      SELECT 
         row_number() OVER (ORDER BY rf.variant_entry_path) AS overall_index,
         rf.variant_entry_path,
         rf.possible_attr_id AS resolved_id,
         rf.attr_val_id,
         rf.numeric_value,
         rf.span_id,
         rf.entity_state_id
      FROM recursive_flattened rf
  ),
  ordered_with_prior AS (
      SELECT o.*,
             (SELECT array_agg(o2.resolved_id ORDER BY o2.overall_index)
              FROM ordered o2
              WHERE o2.overall_index < o.overall_index) AS prior_ids
      FROM ordered o
  )
  SELECT 
      o.overall_index AS "index",
      (
         -- Build address as a JSON array. Each element is either:
         -- a JSON string (if span is NULL) or a JSON array [variant_entry_name, span_label].
         SELECT json_agg(
           CASE
             WHEN (ve_tuple)[1] IS NULL THEN to_json(ve.name)
             ELSE json_build_array(ve.name, s.label)
           END
           ORDER BY i
         )
         FROM generate_subscripts(o.variant_entry_path, 1) AS i
         CROSS JOIN LATERAL (
             SELECT array_agg(x ORDER BY ord) AS ve_tuple
             FROM unnest(o.variant_entry_path[i:i]) WITH ORDINALITY AS t(x, ord)
         ) t
         JOIN variant_entry ve ON ve.id = (ve_tuple)[2]
         LEFT JOIN span s ON s.id = (ve_tuple)[1]
      ) AS address,
      eff.activates,
      eff.effected_by,
      o.numeric_value,
      (
         SELECT s2.label
         FROM span s2
         WHERE s2.id = o.span_id
      ) AS span,
      EXISTS (
         SELECT 1 FROM attr_val_lock l
         WHERE l.locked_attr_val_id = o.attr_val_id
      ) AS locked
  FROM ordered_with_prior o
  CROSS JOIN LATERAL (
      SELECT 
         COALESCE(
           json_agg(f.name ORDER BY f.name) FILTER (
             WHERE e.relationship = 'activates'
               AND f.span_id = o.span_id
               AND o.resolved_id = f.activating_possible_attr_id
               AND EXISTS (
                   SELECT 1 FROM attr_value av
                   WHERE av.possible_attr_id = f.activating_possible_attr_id
                     AND av.entity_state_id = o.entity_state_id
                     AND av.span_id = f.span_id
               )
           ), '[]'::json
         ) AS activates,
         COALESCE(
           json_agg(f.name ORDER BY f.name) FILTER (
             WHERE e.relationship = 'effected_by'
               AND f.span_id = o.span_id
               AND o.resolved_id = f.to_modify_possible_attr_id
               AND f.activating_possible_attr_id = ANY(o.prior_ids)
               AND EXISTS (
                   SELECT 1 FROM attr_value av
                   WHERE av.possible_attr_id = f.activating_possible_attr_id
                     AND av.entity_state_id = o.entity_state_id
                     AND av.span_id = f.span_id
               )
           ), '[]'::json
         ) AS effected_by
      FROM get_effects_from_resolved_path(o.variant_entry_path) e
      JOIN effect f ON f.id = e.id
  ) eff
  ORDER BY o.overall_index;
END;
$$;

## Generate Entity Group

In [ ]:
%%sql

CREATE OR REPLACE PROCEDURE generate_entity_group(
    p_variant_name         VARCHAR,
    p_group_name           VARCHAR,
    p_entity_name_template VARCHAR,
    p_num_entities         INTEGER
)
LANGUAGE plpgsql
AS $$
DECLARE
    v_variant_id       INTEGER;
    v_entity_group_id  INTEGER;
    v_entity_id        INTEGER;
    v_idx              INTEGER;
    v_entity_name      VARCHAR(255);
BEGIN
    -- Look up the variant id using the variant name.
    SELECT id
      INTO v_variant_id
      FROM variant
     WHERE name = p_variant_name;
    IF v_variant_id IS NULL THEN
        RAISE EXCEPTION 'No variant found with name=%', p_variant_name;
    END IF;

    -- Create a new entity group.
    INSERT INTO entity_group (name)
    VALUES (p_group_name)
    RETURNING id INTO v_entity_group_id;

    FOR v_idx IN 1..p_num_entities LOOP
        v_entity_name := format(p_entity_name_template, v_idx);

        -- Create a new entity using the looked-up variant id.
        INSERT INTO entity (variant_id, name)
        VALUES (v_variant_id, v_entity_name)
        RETURNING id INTO v_entity_id;

        INSERT INTO entity_group_link (entity_id, entity_group_id)
        VALUES (v_entity_id, v_entity_group_id);

        -- Call generate_entity_state using the new entity name and time (0 in this example).
        CALL generate_entity_state(v_entity_name, 0);
    END LOOP;
END;
$$;

## Delete Entity Group

In [ ]:
%%sql

CREATE OR REPLACE PROCEDURE delete_entity_group(
    p_group_name      VARCHAR,
    p_delete_entities BOOLEAN DEFAULT false
)
LANGUAGE plpgsql
AS $$
DECLARE
    v_entity_group_id INTEGER;
BEGIN
    -- Retrieve the entity group id based on the given name.
    SELECT id INTO v_entity_group_id
    FROM entity_group
    WHERE name = p_group_name;

    IF NOT FOUND THEN
        RAISE NOTICE 'Entity group "%" not found.', p_group_name;
        RETURN;
    END IF;

    -- If the flag is true, delete all entities that are linked to this group.
    IF p_delete_entities THEN
        DELETE FROM entity
        WHERE id IN (
            SELECT entity_id
            FROM entity_group_link
            WHERE entity_group_id = v_entity_group_id
        );
    END IF;

    -- Delete the entity group. The ON DELETE CASCADE on the foreign key
    -- in entity_group_link will automatically remove any associated links.
    DELETE FROM entity_group
    WHERE id = v_entity_group_id;

    RAISE NOTICE 'Entity group "%" deleted successfully.', p_group_name;
END;
$$;


## Get Entity Group at Time

In [ ]:
%%sql

CREATE OR REPLACE FUNCTION get_entity_group_at_time(
    p_group_name VARCHAR,
    p_time       DOUBLE PRECISION
)
RETURNS TABLE (
    entity_name      VARCHAR,
    entity_state_id  INTEGER,
    address          VARCHAR,
    numeric_value    DOUBLE PRECISION,
    span             VARCHAR,
    activates        VARCHAR,
    effected_by      VARCHAR,
    locked           BOOLEAN
)
LANGUAGE sql
AS $$
WITH eg AS (
    -- Find the entity group by its unique name.
    SELECT id
    FROM entity_group
    WHERE name = p_group_name
),
ent AS (
    -- Get all entities in the group.
    SELECT e.id, e.name
    FROM entity e
    JOIN entity_group_link egl ON e.id = egl.entity_id
    WHERE egl.entity_group_id = (SELECT id FROM eg)
),
est AS (
    -- For each entity, get its entity_state at the given time.
    SELECT e.id AS entity_id, e.name AS entity_name, es.id AS entity_state_id
    FROM ent e
    JOIN entity_state es ON es.entity_id = e.id
    WHERE es."time" = p_time
)
-- For each entity state, call get_entity_state and return one row per state detail.
SELECT
    est.entity_name,
    est.entity_state_id,
    vs.address,
    vs.numeric_value,
    vs.span,
    vs.activates,
    vs.effected_by,
    vs.locked
FROM est
CROSS JOIN LATERAL (
    SELECT *
    FROM get_entity_state(est.entity_name, p_time)
) AS vs;
$$;

## Resolve Causal Path

In [ ]:
%%sql

CREATE OR REPLACE FUNCTION resolve_causal_path(
    p_variant_id INT,
    p_path TEXT
) RETURNS INT
LANGUAGE plpgsql
AS $$
DECLARE
    v_path_parts TEXT[];
    v_current_activable INT := p_variant_id;
    v_activation_id INT;
    v_next_activable INT;
    v_activation_name TEXT;
    v_i INT;
    v_num_parts INT;
    v_parent_causand INT := NULL;
    v_current_causand INT;
    v_root INT;
BEGIN
    -- Split the provided slash-separated path into an array of activation names.
    v_path_parts := string_to_array(p_path, '/');
    v_num_parts := array_length(v_path_parts, 1);
    IF v_num_parts IS NULL OR v_num_parts = 0 THEN
        RAISE EXCEPTION 'Activation path is empty';
    END IF;

    FOR v_i IN 1..v_num_parts LOOP
        v_activation_name := v_path_parts[v_i];

        -- Look up the activation record for the current activable.
        SELECT id, to_activable
          INTO v_activation_id, v_next_activable
          FROM activation
         WHERE from_activable = v_current_activable
           AND name = v_activation_name
         LIMIT 1;
        IF NOT FOUND THEN
            RAISE EXCEPTION 'Activation "%" not found from activable % at path index %',
                            v_activation_name, v_current_activable, v_i;
        END IF;

        IF v_parent_causand IS NULL THEN
            -- For the first activation, check if a causand already exists with:
            --    - the given activation,
            --    - a cause record having root = p_variant_id,
            --    - and the same activation name.
            SELECT c.id
              INTO v_current_causand
              FROM causand c
              JOIN cause cs ON cs.causand = c.id
             WHERE c.activation = v_activation_id
               AND cs.root = p_variant_id
               AND cs.name = v_activation_name
             LIMIT 1;
            IF NOT FOUND THEN
                -- Not found: create new causand and cause record.
                INSERT INTO causand (activation)
                VALUES (v_activation_id)
                RETURNING id INTO v_current_causand;
                INSERT INTO cause (causand, root, name)
                VALUES (v_current_causand, p_variant_id, v_activation_name);
            END IF;
        ELSE
            -- For subsequent activations, look for an existing causand node that is:
            --    - associated with the activation,
            --    - has a cause record with the given activation name,
            --    - and is linked as a child (via antecedent) from the previous causand.
            SELECT c.id
              INTO v_current_causand
              FROM causand c
              JOIN cause cs ON cs.causand = c.id
              JOIN antecedent ant ON ant.to_causand = c.id
             WHERE c.activation = v_activation_id
               AND cs.name = v_activation_name
               AND ant.from_causand = v_parent_causand
             LIMIT 1;
            IF NOT FOUND THEN
                INSERT INTO causand (activation)
                VALUES (v_activation_id)
                RETURNING id INTO v_current_causand;
                -- Link the new causand node to its parent.
                INSERT INTO antecedent (from_causand, to_causand)
                VALUES (v_parent_causand, v_current_causand);
                -- Use the parent's root value for the new node.
                SELECT cs.root INTO v_root FROM cause cs WHERE cs.causand = v_parent_causand;
                INSERT INTO cause (causand, root, name)
                VALUES (v_current_causand, v_root, v_activation_name);
            END IF;
        END IF;

        -- Prepare for next iteration.
        v_parent_causand := v_current_causand;
        v_current_activable := v_next_activable;
    END LOOP;

    RETURN v_current_causand;
END;
$$;

## Get Causal Chain

In [ ]:
%%sql

CREATE OR REPLACE FUNCTION get_causal_chain(
    p_causand_id INT
) RETURNS TABLE(level INT, causand_id INT, activation_name CITEXT)
LANGUAGE plpgsql
AS $$
BEGIN
  RETURN QUERY 
  WITH RECURSIVE rev_chain AS (
    -- Start at the provided causand (final node in the chain)
    SELECT 
      c.id AS causand_id, 
      c.activation, 
      1 AS rlevel
    FROM causand c
    WHERE c.id = p_causand_id

    UNION ALL

    -- Walk backwards: if the current node’s activation appears as a successor,
    -- then fetch the causand whose activation is the corresponding node.
    SELECT 
      c_prev.id,
      c_prev.activation,
      rc.rlevel + 1
    FROM rev_chain rc
    JOIN activation_successor asuc ON asuc.successor = rc.activation
    JOIN causand c_prev ON c_prev.activation = asuc.node
  ),
  max_level AS (
    SELECT MAX(rlevel) AS max_r FROM rev_chain
  )
  SELECT 
    (m.max_r - rc.rlevel + 1) AS level,
    rc.causand_id,
    cs.name AS activation_name
  FROM rev_chain rc, max_level m
  JOIN cause cs ON cs.causand = rc.causand_id
  ORDER BY level;
END;
$$;

## Compare Causal Order

In [ ]:
%%sql

CREATE OR REPLACE FUNCTION compare_causal_order(
    p_a INT,
    p_b INT
) RETURNS INT
LANGUAGE plpgsql
AS $$
DECLARE
    rec RECORD;
BEGIN
    FOR rec IN
        SELECT COALESCE(a.level, b.level) AS level,
               a.activation_name AS a_name,
               b.activation_name AS b_name
          FROM get_causal_chain(p_a) a
          FULL OUTER JOIN get_causal_chain(p_b) b ON a.level = b.level
          ORDER BY level
    LOOP
        -- If one chain is shorter, treat the missing value as "less".
        IF rec.a_name IS NULL AND rec.b_name IS NOT NULL THEN
            RETURN -1;
        ELSIF rec.b_name IS NULL AND rec.a_name IS NOT NULL THEN
            RETURN 1;
        ELSIF rec.a_name < rec.b_name THEN
            RETURN -1;
        ELSIF rec.a_name > rec.b_name THEN
            RETURN 1;
        END IF;
        -- Otherwise, they are equal at this level; continue.
    END LOOP;
    RETURN 0;
END;
$$;

## Get Variant ID

In [ ]:
%%sql

CREATE OR REPLACE FUNCTION get_variant_id(
    p_variant_name VARCHAR
) RETURNS INTEGER
LANGUAGE plpgsql
AS $$
DECLARE
    variant_id INTEGER;
BEGIN
    SELECT id
      INTO variant_id
      FROM variant
     WHERE name = p_variant_name;
     
    IF NOT FOUND THEN
        RAISE EXCEPTION 'Variant "%" not found', variant_id;
    END IF;
    
    RETURN variant_id;
END;
$$;

## Get Attribute ID

In [ ]:
%%sql

CREATE OR REPLACE FUNCTION get_attribute_id(
    p_attribute_name VARCHAR
) RETURNS INTEGER
LANGUAGE plpgsql
AS $$
DECLARE
    attribute_id INTEGER;
BEGIN
    SELECT id
      INTO attribute_id
      FROM attribute
     WHERE name = p_attribute_name;
     
    IF NOT FOUND THEN
        RAISE EXCEPTION 'Attribute "%" not found', p_attribute_name;
    END IF;
    
    RETURN attribute_id;
END;
$$;

## Get Span ID

In [ ]:
%%sql

CREATE OR REPLACE FUNCTION get_span_id(
    p_attribute_name VARCHAR,
    p_span_type      attr_type,
    p_span_label     VARCHAR
) RETURNS INTEGER
LANGUAGE plpgsql
AS $$
DECLARE
    span_id INTEGER;
BEGIN
    SELECT s.id
      INTO span_id
      FROM span s
      JOIN attribute a ON a.id = s.attribute_id
     WHERE s.label = p_span_label
       AND a.name = p_attribute_name
       AND s.type = p_span_type;

    IF NOT FOUND THEN
        RAISE EXCEPTION 'Span on attribute "%" of type "%" and label "%" not found', p_attribute_name, p_span_type, p_span_label;
    END IF;

    RETURN span_id;
END;
$$;

## Get Effect ID

In [ ]:
%%sql

CREATE OR REPLACE FUNCTION get_effect_id(
    p_variant_name   VARCHAR,
    p_effect_name    VARCHAR
) RETURNS INTEGER
LANGUAGE plpgsql
AS $$
DECLARE
    effect_id INTEGER;
BEGIN
    SELECT e.id
      INTO effect_id
      FROM effect e
      JOIN variant vt ON vt.id = e.variant_id
     WHERE vt.name = p_variant_name
       AND e.name = p_effect_name;

    IF NOT FOUND THEN
        RAISE EXCEPTION 'effect "%" in variant "%" not found', p_effect_name, p_variant_name;
    END IF;

    RETURN effect_id;
END;
$$;

## Add Continuous Effect

In [ ]:
%%sql

CREATE OR REPLACE PROCEDURE add_continuous_effect(
    p_effect_name              VARCHAR,
    p_variant_name             VARCHAR,
    p_activating_span_path     JSON,  -- now a JSON array (e.g. [ [variant, span_label], ... ])
    p_target_variant_attr_path JSON,  -- same format
    p_delta_mode               DOUBLE PRECISION,
    p_delta_conc               DOUBLE PRECISION,
    p_delta_skew               DOUBLE PRECISION
)
LANGUAGE plpgsql
AS $$
DECLARE
    v_variant_id                  INT;
    v_activating_possible_attr_id INT;
    v_to_modify_possible_attr_id  INT;
    v_to_modify_attr_type         attr_type;
    v_triggering_span_id          INT;
    v_effect_id                   INT;
    v_triggering_span_label       VARCHAR;
    v_chain_length                INT;
BEGIN
    -- Extract triggering span label from the last element of the activating path.
    v_chain_length := json_array_length(p_activating_span_path);
    IF v_chain_length = 0 THEN
         RAISE EXCEPTION 'Activating span is empty';
    END IF;
    v_triggering_span_label := (p_activating_span_path->(v_chain_length - 1))->>1;

    v_variant_id := get_variant_id(p_variant_name);
    v_activating_possible_attr_id := resolve_possible_attr_path(v_variant_id, p_activating_span_path);
    v_to_modify_possible_attr_id  := resolve_possible_attr_path(v_variant_id, p_target_variant_attr_path);

    -- Ensure the attribute is continuous.
    SELECT a.type
      INTO v_to_modify_attr_type
      FROM possible_attr pa
      JOIN attribute_ref ar ON ar.variant_entry_id = pa.variant_entry_id
      JOIN attribute a ON a.id = ar.attribute_id
     WHERE pa.id = v_to_modify_possible_attr_id
     LIMIT 1;
    IF v_to_modify_attr_type <> 'continuous' THEN
         RAISE EXCEPTION 'The attribute associated with to_modify_possible_attr_id (%) is %, but expected continuous',
                         v_to_modify_possible_attr_id, v_to_modify_attr_type;
    END IF;

    -- Resolve the triggering span ID.
    SELECT s.id
      INTO v_triggering_span_id
      FROM span s
      WHERE s.label = v_triggering_span_label
        AND s.attribute_id = (
              SELECT ar.attribute_id
                FROM possible_attr pa
                JOIN attribute_ref ar ON ar.variant_entry_id = pa.variant_entry_id
               WHERE pa.id = v_activating_possible_attr_id
              LIMIT 1
          )
      LIMIT 1;
    IF NOT FOUND THEN
       RAISE EXCEPTION 'Triggering span with label "%" not found for the activating branch', v_triggering_span_label;
    END IF;

    -- Insert the effect record.
    INSERT INTO effect(
        name, 
        variant_id, 
        span_id, 
        to_modify_possible_attr_id, 
        activating_possible_attr_id
    )
    VALUES (
        p_effect_name,
        v_variant_id,
        v_triggering_span_id,
        v_to_modify_possible_attr_id,
        v_activating_possible_attr_id
    )
    RETURNING id INTO v_effect_id;

    -- Insert continuous effect details.
    INSERT INTO delta_param_eft(effect_id, delta_mode, delta_conc, delta_skew)
    VALUES (v_effect_id, p_delta_mode, p_delta_conc, p_delta_skew);
END;
$$;

## Add Discrete Effect

In [ ]:
%%sql

CREATE OR REPLACE PROCEDURE add_discrete_effect(
    p_effect_name              VARCHAR,
    p_variant_name             VARCHAR,
    p_activating_span_path     JSON,  -- now a JSON array (e.g. [ [variant, span_label], ... ])
    p_target_variant_attr_path JSON,  -- same format
    p_changes                  JSON,  -- Expecting an array of tuples: [[label, weight], ...]
    p_remove_unmentioned       BOOLEAN DEFAULT FALSE
)
LANGUAGE plpgsql
AS $$
DECLARE
    v_variant_id                  INT;
    v_activating_possible_attr_id INT;
    v_to_modify_possible_attr_id  INT;
    v_triggering_span_id          INT;
    v_effect_id                   INT;
    v_attribute_id                INT;
    v_attribute_name              VARCHAR;
    v_to_modify_attr_type         attr_type;
    v_span_id                     INT;
    rec_json                      JSON;
    rec_record                    RECORD;
    v_label                       TEXT;
    v_weight                      DOUBLE PRECISION;
    v_processed_labels            TEXT[] := '{}';
    v_triggering_span_label       VARCHAR;
    v_chain_length                INT;
BEGIN
    -- Extract triggering span label from the last element of the activating path.
    v_chain_length := json_array_length(p_activating_span_path);
    IF v_chain_length = 0 THEN
         RAISE EXCEPTION 'Activating span path is empty';
    END IF;
    v_triggering_span_label := (p_activating_span_path->(v_chain_length - 1))->>1;

    v_variant_id := get_variant_id(p_variant_name);
    v_activating_possible_attr_id := resolve_possible_attr_path(v_variant_id, p_activating_span_path);
    v_to_modify_possible_attr_id  := resolve_possible_attr_path(v_variant_id, p_target_variant_attr_path);

    -- Resolve the triggering span ID.
    SELECT s.id 
      INTO v_triggering_span_id
      FROM span s
      WHERE s.label = v_triggering_span_label
        AND s.attribute_id = (
              SELECT ar.attribute_id
                FROM possible_attr pa
                JOIN attribute_ref ar ON ar.variant_entry_id = pa.variant_entry_id
               WHERE pa.id = v_activating_possible_attr_id
              LIMIT 1
          )
      LIMIT 1;
    IF NOT FOUND THEN
       RAISE EXCEPTION 'Triggering span with label "%" not found for the activating branch', v_triggering_span_label;
    END IF;

    -- Insert the effect record.
    INSERT INTO effect(
        name, 
        variant_id, 
        span_id, 
        to_modify_possible_attr_id, 
        activating_possible_attr_id
    )
    VALUES (
        p_effect_name,
        v_variant_id,
        v_triggering_span_id,
        v_to_modify_possible_attr_id,
        v_activating_possible_attr_id
    )
    RETURNING id INTO v_effect_id;

    -- Retrieve the attribute for the to-modify branch.
    SELECT a.id, a.name, a.type
      INTO v_attribute_id, v_attribute_name, v_to_modify_attr_type
      FROM possible_attr pa
      JOIN attribute_ref ar ON ar.variant_entry_id = pa.variant_entry_id
      JOIN attribute a ON a.id = ar.attribute_id
     WHERE pa.id = v_to_modify_possible_attr_id
     LIMIT 1;

    -- Ensure the attribute type is 'discrete'.
    IF v_to_modify_attr_type <> 'discrete' THEN
         RAISE EXCEPTION 'The attribute associated with to_modify_possible_attr_id (%) is %, but expected discrete',
                         v_to_modify_possible_attr_id, v_to_modify_attr_type;
    END IF;

    -- Process each change tuple in p_changes.
    IF p_changes IS NOT NULL THEN
       FOR rec_json IN SELECT * FROM json_array_elements(p_changes) LOOP
          v_label  := rec_json->>0;
          v_weight := (rec_json->>1)::double precision;
          v_processed_labels := array_append(v_processed_labels, v_label);

          SELECT id 
            INTO v_span_id
            FROM span
           WHERE attribute_id = v_attribute_id
             AND label = v_label
             AND type = 'discrete'
           LIMIT 1;

          IF NOT FOUND THEN
             INSERT INTO span(
                 attribute_id, 
                 label, 
                 type, 
                 is_pinned, 
                 weight, 
                 max_value, 
                 min_value
             )
             VALUES (v_attribute_id, v_label, 'discrete', false, v_weight, NULL, NULL)
             RETURNING id INTO v_span_id;

             INSERT INTO add_span_eft(effect_id, span_id)
             VALUES (v_effect_id, v_span_id);
          ELSE
             IF v_weight = 0 THEN
                INSERT INTO remove_span_eft(effect_id, span_id)
                VALUES (v_effect_id, v_span_id);
             ELSE
                INSERT INTO delta_weight_eft(effect_id, span_id, delta_weight)
                VALUES (v_effect_id, v_span_id, v_weight);
             END IF;
          END IF;
       END LOOP;
    END IF;

    -- Remove any unmentioned spans if requested.
    IF p_remove_unmentioned THEN
       FOR rec_record IN
         SELECT id, label 
         FROM span
         WHERE attribute_id = v_attribute_id
           AND type = 'discrete'
           AND NOT (label = ANY(v_processed_labels))
       LOOP
         INSERT INTO remove_span_eft(effect_id, span_id)
         VALUES (v_effect_id, rec_record.id);
       END LOOP;
    END IF;
END;
$$;

## Remove Effect

In [ ]:
%%sql

CREATE OR REPLACE PROCEDURE remove_effect(
    p_effect_name VARCHAR
)
LANGUAGE plpgsql
AS $$
DECLARE
    v_effect_id INT;
    v_deleted_count INT;
BEGIN
    -- Look up the effect by its unique name.
    SELECT id
      INTO v_effect_id
      FROM effect
     WHERE name = p_effect_name
     LIMIT 1;
     
    IF NOT FOUND THEN
       RAISE NOTICE 'Effect "%" not found. Nothing to remove.', p_effect_name;
       RETURN;
    END IF;

    -- Delete the effect record.
    DELETE FROM effect
     WHERE id = v_effect_id;
    
    GET DIAGNOSTICS v_deleted_count = ROW_COUNT;
    
    IF v_deleted_count > 0 THEN
       RAISE NOTICE 'Effect "%" removed successfully.', p_effect_name;
    ELSE
       RAISE NOTICE 'No effect was removed for "%".', p_effect_name;
    END IF;
END;
$$;

# Inspect Schema

## List Tables

In [ ]:
%%sql

SELECT tablename AS name
FROM pg_catalog.pg_tables
WHERE schemaname NOT IN ('pg_catalog', 'information_schema')
ORDER BY schemaname, tablename;

## List Procedures

In [ ]:
%%sql

SELECT proname AS name, pg_catalog.pg_get_function_arguments(p.oid) AS arguments
FROM pg_catalog.pg_proc p
JOIN pg_catalog.pg_namespace n ON n.oid = p.pronamespace
WHERE p.prokind = 'p' AND n.nspname = 'public'
ORDER BY n.nspname, proname;

## List Functions

In [ ]:
%%sql

SELECT proname AS name,
       pg_catalog.pg_get_function_arguments(p.oid) AS arguments,
       pg_catalog.pg_get_function_result(p.oid) AS return_type
FROM pg_catalog.pg_proc p
JOIN pg_catalog.pg_namespace n ON n.oid = p.pronamespace
WHERE p.prokind = 'f' AND n.nspname = 'public'
ORDER BY n.nspname, proname;

## List Indices

In [ ]:
%%sql

SELECT i.relname AS index_name, t.relname AS table_name
FROM pg_catalog.pg_index ix
JOIN pg_catalog.pg_class i ON i.oid = ix.indexrelid
JOIN pg_catalog.pg_class t ON t.oid = ix.indrelid
JOIN pg_catalog.pg_namespace n ON n.oid = t.relnamespace
WHERE n.nspname = 'public'
ORDER BY n.nspname, t.relname, i.relname;

# Testing

## Gender Attribute

### Delete Gender Attribute

In [ ]:
%%sql

CALL delete_discrete_attribute('Gender');

### Add Gender Attribute

In [ ]:
%%sql

call add_discrete_attribute('Gender', '[
    ["Male", 1], 
    ["Female", 1]
]');

### View Gender Attribute

In [ ]:
%%sql

SELECT * FROM get_discrete_attribute('Gender');

## Hair Color Attribute

### Delete Hair Color Attribute

In [ ]:
%%sql

CALL delete_discrete_attribute('Hair Color');

### Add Hair Color Attribute

In [ ]:
%%sql

call add_discrete_attribute('Hair Color', '[
    ["Blonde", 2], 
    ["Brown", 3],
    ["Golden", 2],
    ["Auburn", 2],
    ["Ginger", 1],
    ["Sandy", 2],
    ["Black", 3],
    ["Gray", 2]
]');

### View Hair Color Attribute

In [ ]:
%%sql

SELECT * FROM get_discrete_attribute('Hair Color');

## Height Attribute

### Delete Height Attribute

In [ ]:
%%sql

CALL delete_continuous_attribute('Height');

### Add Height Attribute

In [ ]:
%%sql

call add_continuous_attribute('Height', 0, 68, 96, 32, 0, 4, 'in.', '[
    ["Short", 60],
    ["Average", 74],
    ["Tall", 96]
]');

### View Height Attribute

In [ ]:
%%sql

SELECT * FROM get_continuous_attribute('Height');

## Human Variant

### Delete

In [ ]:
%%sql

CALL delete_variant('Human');

### Add

In [ ]:
%%sql

CALL add_variant('Human', '[
    ["a", "Gender"],
    ["a", "Hair Color"],
    ["a", "Height"]
]');

### Get

In [ ]:
%%sql

SELECT * FROM get_variant('Human', 5);

## Generate a Group of Humans

### Delete

In [ ]:
%%sql

CALL delete_entity_group('Test Humans 1', true);

### Create

In [ ]:
%%sql

CALL generate_entity_group('Human', 'Test Humans 1', 'Test Human %s', 10);

### View

In [ ]:
%%sql

SELECT * FROM get_entity_group_at_time('Test Humans 1', 0);

# The Robot Test

## Paint Attribute

In [ ]:
%%sql

CALL add_discrete_attribute('Color', '[
        ["None", 3],
        ["Red", 2],
        ["Yellow", 2],
        ["Green", 2],
        ["Blue", 2],
        ["Orange", 2],
        ["White", 2],
        ["Black", 2],
        ["Purple", 2],
        ["Custom", 1, "Custom Pigment"]
]');

In [ ]:
%%sql

CALL add_continuous_attribute('R', 0, 127, 255, 0, 0, 0, NULL, NULL);

In [ ]:
%%sql

CALL add_continuous_attribute('G', 0, 127, 255, 0, 0, 0, NULL, NULL);

In [ ]:
%%sql

CALL add_continuous_attribute('B', 0, 127, 255, 0, 0, 0, NULL, NULL);

In [ ]:
%%sql

CALL add_variant('Custom Pigment', '[
    ["a", "R"], 
    ["a", "G"], 
    ["a", "B"]
]');

In [ ]:
%%sql

CALL add_discrete_attribute(
    'Sheen', '[
        ["Glossy", 3],
        ["Egg-Shell", 1],
        ["Matte", 2],
        ["Semi-Gloss", 2]
]');

In [ ]:
%%sql

CALL add_variant('Paint', '[
    ["a", "Color"], 
    ["a", "Sheen"]
]');

## Material Attribute

In [ ]:
%%sql

CALL add_discrete_attribute(
    'Material', '[
        ["Steel", 4],
        ["Aluminum", 6],
        ["Bronze", 2],
        ["Iron", 2],
        ["Carbon-Fiber", 6],
        ["PVC", 4],
        ["Animal Bone", 1],
        ["Wood", 2]
    ]'
);

## Power-Level Attribute

In [ ]:
%%sql

CALL add_continuous_attribute('Power-Level', 0, 50, 100, 24, 0, 2, NULL, '[
    ["Very Weak", 20],
    ["Weak", 40],
    ["Normal", 60],
    ["Strong", 80],
    ["Ultra", 100]
]');

## Length Attribute

In [ ]:
%%sql

CALL add_continuous_attribute('Length', .01, 36, 120, 36, 0, 2, 'in.', NULL);

## Diameter Attribute

In [ ]:
%%sql

CALL add_continuous_attribute('Diameter', .01, 1, 4, 12, 0, 2, 'in.', NULL);

## Sub-Arm Attribute

In [ ]:
%%sql

CALL add_discrete_attribute('Sub-Arm', '[["No", 4], ["Yes", 1, "Arm"]]');

## Attachment Attribute

In [ ]:
%%sql

CALL add_discrete_attribute(
    'Attachment', '[
        ["Cutter", 3],
        ["Welder", 2],
        ["Pincher", 3],
        ["Grabber", 3],
        ["Laser", 1],
        ["Cannon", 1],
        ["Hammer", 2],
        ["Grappler", 1],
        ["Vacuum", 2]
    ]'
);

## Input Attribute

In [ ]:
%%sql

CALL add_discrete_attribute(
    'Input', '[
        ["Camera", 5],
        ["Microphone", 5],
        ["Smell-O-Sense", 2],
        ["Infra-Red", 3],
        ["X-Ray", 3],
        ["GPS", 4],
        ["Temporal", 1]
    ]'
);

## Output Attribute

In [ ]:
%%sql

CALL add_discrete_attribute(
    'Output', '[
        ["MIDI Tone", 5],
        ["Text Screen", 7],
        ["Clicker", 4],
        ["Recorded Phrases", 7],
        ["X-Ray", 5],
        ["Light Array", 5],
        ["Party Blower", 1],
        ["Kazoo", 1]
    ]'
);

## Foot Attribute

In [ ]:
%%sql

CALL add_discrete_attribute(
    'Foot', '[
        ["Roller Skate", 5],
        ["Stomper", 7],
        ["Tread", 4],
        ["Pogo", 7],
        ["Thruster", 5],
        ["Suction", 5]
    ]'
);

## Shape Attribute

In [ ]:
%%sql

CALL add_discrete_attribute(
    'Shape', '[
        ["Spherical", 5],
        ["Blocky", 7],
        ["Sculpted", 4],
        ["Spiky", 2],
        ["Segmented", 5],
        ["Donut", 1]
    ]'
);

## Power Source Attribute

In [ ]:
%%sql

CALL add_discrete_attribute(
    'Power Source', '[
        ["Battery", 15],
        ["Solar", 10],
        ["Diesel", 10],
        ["Wired", 12],
        ["Wind", 4],
        ["Nuclear Core", 4]
    ]'
);

## Neck Width Attribute

In [ ]:
%%sql

CALL add_continuous_attribute('Neck Width', 1, 10, 2, 2, 0, 1, 'in.', NULL);

## Weight Attribute

In [ ]:
%%sql

CALL add_continuous_attribute('Weight', .01, 20, 5000, 2, 0, 1, 'lbs.', NULL);

## Torso Variant

In [ ]:
%%sql

CALL add_variant('Torso', '[
    ["a", "Shape"], 
    ["a", "Material"], 
    ["a", "Power-Level"], 
    ["v", "Paint"]
]');

## Head Variant

In [ ]:
%%sql

CALL add_variant('Head', '[
    ["a", "Input"], 
    ["a", "Output"], 
    ["a", "Material"], 
    ["a", "Power-Level"], 
    ["v", "Paint"]
]');

## Leg Variant

In [ ]:
%%sql

CALL add_variant('Leg', '[
    ["a", "Foot"], 
    ["a", "Material"], 
    ["a", "Power-Level"], 
    ["a", "Length"],
    ["a", "Diameter"],
    ["v", "Paint"]
]');

## Hand Variant

In [ ]:
%%sql

-- Create the variant groupings
CALL add_variant('Hand', '[
    ["a", "Attachment"], 
    ["a", "Material"],  
    ["a", "Power-Level"], 
    ["v", "Paint"]
]');

## Arm Variant

In [ ]:
%%sql

CALL add_variant('Arm', '[
    ["v", "Hand"], 
    ["a", "Material"], 
    ["v", "Paint"], 
    ["a", "Length"], 
    ["a", "Diameter"], 
    ["a", "Sub-Arm"]
]');

## Robot Variant

In [ ]:
%%sql

CALL add_variant('Robot', '[
    ["a", "Power Source"],
    ["v", "Head"],
    ["a", "Neck Width"],
    ["v", "Torso"],
    ["v", "Arm", "Left Arm"], 
    ["v", "Arm", "Right Arm"], 
    ["v", "Leg", "Left Leg"], 
    ["v", "Leg", "Right Leg"],
    ["a", "Weight"]
]');

## Effects

### Welder hands cannot be made of wood'

In [ ]:
%%sql

-- Add effects

-- Test: discrete effect on same variant level, disable span
CALL add_discrete_effect(
    'Welder hands cannot be made of wood',
    'Hand',
    '[["Attachment", "Welder"]]',
    '["Material"]',
    '[["Wood", 0]]'
);

### Wooden hands have lower power level

In [ ]:
%%sql

-- Test: continous effect on same variant level
CALL add_continuous_effect(
    'Wooden hands have lower power level',
    'Hand',
    '[["Material", "Wood"]]',
    '["Power-Level"]',
    -20, 0, 0
);

### Spherical torsos are more likely to be painted white or blue

In [ ]:
%%sql

-- Test: stacked discrete effect on an attribute one level down
CALL add_discrete_effect(
    'Spherical torsos are more likely to be painted white or blue',
    'Torso',
    '[["Shape", "Spherical"]]',
    '["Paint", "Color"]',
    '[["White", 10], ["Blue", 5]]'
);

### Animal bone torsos are more likely to be painted white or green

In [ ]:
%%sql

CALL add_discrete_effect(
    'Animal bone torsos are more likely to be painted white or green',
    'Torso',
    '[["Material", "Animal Bone"]]',
    '["Paint", "Color"]',
    '[["White", 10], ["Green", 5]]'
);

### Wooden torsos have lower power level

In [ ]:
%%sql

-- Test: stacked continuous effect on same variant level
CALL add_continuous_effect(
    'Wooden torsos have lower power level',
    'Torso',
    '[["Material", "Wood"]]',
    '["Power-Level"]',
    -20, 0, 0
);

### Blocky torsos have higher power level

In [ ]:
%%sql

CALL add_continuous_effect(
    'Blocky torsos have higher power level',
    'Torso',
    '[["Shape", "Blocky"]]',
    '["Power-Level"]',
    15, 0, 0
);

### All diesel-powered robots have a roller skate on their left foot

In [ ]:
%%sql

-- Test: discrete effect on an attribute one level down, restricting options
CALL add_discrete_effect(
    'All diesel-powered robots have a roller skate on their left foot',
    'Robot',
    '[["Power Source", "Diesel"]]',
    '["Left Leg", "Foot"]',	
    '[["Roller Skate", 1]]',
    true
);

### Robot with wired power are less likely to have heads with camera input

In [ ]:
%%sql

-- Test: one level down
CALL add_discrete_effect(
    'Wired-powered robots are less likely to have camera-input heads',
    'Robot',
    '[["Power Source", "Wired"]]',
    '["Head", "Input"]',	
    '[["Camera", -3]]',
    true
);

### Robots with camera input heads are more likely to have wider necks

In [ ]:
%%sql

-- Test: continuous effect on an attribute multiple levels down (including recursive)
CALL add_continuous_effect(
    'Robots with camera input heads are more likely to have wider necks',
    'Robot',
    '["Head", ["Input", "Camera"]]',
    '["Neck Width"]',
    5, 0, 0
);

### Robots with nuclear cores have high-powered heads

In [ ]:
%%sql

-- Test: continuous effect on an attribute multiple levels down (including recursive)
CALL add_continuous_effect(
    'Robots with nuclear cores have high-powered heads',
    'Robot',
    '[["Power Source", "Nuclear Core"]]',
    '["Head", "Power-Level"]',
    90, 6, 4
);

### Robots with nuclear cores have high-powered torsos

In [ ]:
%%sql

CALL add_continuous_effect(
    'Robots with nuclear cores have high-powered torsos',
    'Robot',
    '[["Power Source", "Nuclear Core"]]',
    '["Torso", "Power-Level"]',
    90, 6, 4
);

### Robots with nuclear cores have high-powered left legs

In [ ]:
%%sql

CALL add_continuous_effect(
    'Robots with nuclear cores have high-powered left legs',
    'Robot',
    '[["Power Source", "Nuclear Core"]]',
    '["Left Leg", "Power-Level"]',
    90, 6, 4
);

### Robots with nuclear cores have high-powered right legs

In [ ]:
%%sql

CALL add_continuous_effect(
    'Robots with nuclear cores have high-powered right legs',
    'Robot',
    '[["Power Source", "Nuclear Core"]]',
    '["Right Leg", "Power-Level"]',
    90, 6, 4
);

### Robots with nuclear cores have high-powered right hands

In [ ]:
%%sql

CALL add_continuous_effect(
    'Robots with nuclear cores have high-powered right hands',
    'Robot',
    '[["Power Source", "Nuclear Core"]]',
    '["Right Arm", "Hand", "Power-Level"]',
    90, 6, 4
);

### Robots with nuclear cores have high-powered left hands

In [ ]:
%%sql

CALL add_continuous_effect(
    'Robots with nuclear cores have high-powered left hands',
    'Robot',
    '[["Power Source", "Nuclear Core"]]',
    '["Left Arm", "Hand", "Power-Level"]',
    90, 6, 4
);

### Robots with nuclear cores have high-powered right sub-arm hands

In [ ]:
%%sql

CALL add_continuous_effect(
    'Robots with nuclear cores have high-powered right sub-arm hands',
    'Robot',
    '[["Power Source", "Nuclear Core"]]',
    '["Right Arm", ["Sub-Arm", "Yes"], "Hand", "Power-Level"]',
    90, 6, 4
);

### Robots with nuclear cores have high-powered left sub-arm hands

In [ ]:
%%sql

CALL add_continuous_effect(
    'Robots with nuclear cores have high-powered left sub-arm hands',
    'Robot',
    '[["Power Source", "Nuclear Core"]]',
    '["Left Arm", ["Sub-Arm", "Yes"], "Hand", "Power-Level"]',
    90, 6, 4
);

### Robots with steel torsos have a higher weight

In [ ]:
%%sql

-- Test: continous effect on an attribute one level up
CALL add_continuous_effect(
    'Robots with steel torsos have a higher weight',
    'Robot',
    '["Torso", ["Material", "Steel"]]',
    '["Weight"]',
    50, 0, 2
);

### Robots with iron torsos have a higher weight

In [ ]:
%%sql

CALL add_continuous_effect(
    'Robots with iron torsos have a higher weight',
    'Robot',
    '["Torso", ["Material", "Iron"]]',
    '["Weight"]',
    100, 0, 3
);

### Grappler hands are more likely to be on metal arms

In [ ]:
%%sql

-- Test: discrete effect on an attribute one level up
CALL add_discrete_effect(
    'Grappler hands are more likely to be on metal arms',
    'Arm',
    '["Hand", ["Attachment", "Grappler"]]',
    '["Material"]',
    '[["Steel", 5],
      ["Aluminum", 5],
      ["Bronze", 5],
      ["Iron", 5],
      ["Carbon-Fiber", 5]]',
    true
);

### Heads with kazoo outputs can be painted rainbow

In [ ]:
%%sql

-- Test: discrete effect that adds new spans
CALL add_discrete_effect(
    'Heads with kazoo outputs can be painted rainbow',
    'Head',
    '[["Output", "Kazoo"]]',
    '["Paint", "Color"]',
    '[["Rainbow", 100]]'
);

### Get Variant

In [ ]:
%%sql

SELECT * FROM get_variant('Robot', 4);

### Generate Robots

In [ ]:
%%sql

CALL generate_entity_group('Robot', 'Test Robots 1', 'Robot %s', 10);

### Inspect Output

In [ ]:
%%sql

SELECT * FROM get_entity_group_at_time('Test Robots 1', 0);